In [ ]:
import sys
sys.path.append('/import/silo2/aloe8475/Documents/edamame')

In [ ]:
from scipy.io import loadmat, savemat
from scipy.stats import kurtosis, skew
from scipy.spatial import distance
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import datetime
import networkx as nx
from edamame import *
from tqdm.notebook import tqdm
import os
import edamame.core.wires as wires
from random import choice
from IPython.core.debugger import set_trace
# import nct
import bct
import pandas as pd

import pickle 
import _pickle as cPickle
import bz2

def compressed_pickle(obj, filename,protocol=-1):
    with gzip.open(filename, 'wb') as f:
        cPickle.dump(obj, f, protocol)
def decompress_pickle(file):
    with gzip.open(file, 'rb') as f:
        loaded_object = cPickle.load(f,encoding="bytes") #change this for python 2
        return loaded_object


In [ ]:
def connected_component_subgraphs(G):
    for c in nx.connected_components(G):
        yield G.subgraph(c)

In [ ]:
#Define Save Location
saveLoc='/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/Network Comparison/'
saveFig='/import/silo2/aloe8475/Documents/CODE/Data/Figures/Functional Connectivity/'
saveResults='/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/'

### Elegans:

In [ ]:
#LOAD:
name=r'/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks/barpartite_LinearTransformation_2V.pkl'
print('Loading Crossbar')
file = open(name, 'rb')
[crossbar] = pickle.load(file)
name=r'/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks/elegans_LinearTransformation_2V.pkl'
print('Loading Elegans Networks')
file = open(name, 'rb')
[Elegans] = pickle.load(file)

#### Run Tasks:

In [ ]:
onAmp=[0.2,0.5,0.75,1,1.25,1.5,1.75,2,3,5,10]#np.linspace(0,10,201)

In [ ]:
def nonLinearTrans_specificNodes(network, targetType, nodesList, repeats = 1,time=10,dt=1e-3):
    """
    targetType can be DoubleFrequency and all other possible biasType.
    wireVoltage[T,N] can also be the first positional parameter.
    """
    from edamame.core.simSettings import stimulus__

    if targetType == 'DoubleFrequency':
        target = stimulus__(network.stimulus[0].biasType, network.TimeVector, f = 2*network.stimulus[0].f).signal
    else:
        target = stimulus__(targetType, dt=dt,T=time, f = network.stimulus[0].f).signal
        
    wireVoltage = network.wireVoltage    
    T, N = wireVoltage.shape
    output = dict()
    accuracy = np.zeros(repeats)
    weight = np.zeros((repeats, len(nodesList)+1))
    result = np.zeros((repeats, T))
    for i in range(repeats):
        lhs = np.hstack((np.ones((T, 1)), wireVoltage[:,nodesList]))
        weight[i,:] = np.linalg.lstsq(lhs, target, rcond = None)[0]
#         set_trace()
        result[i,:] = np.dot(lhs, weight[i,:])
        accuracy[i] = 1-getRNMSE(result[i,:], target)
        
    output['weight'] = np.mean(weight, axis = 0)
    output['result'] = np.mean(result, axis = 0)
    output['accuracy'] = np.mean(accuracy)
    output['target'] = target
    
    return output 

In [ ]:
def getNLT_Elegans(Graph=[],amp=2,nwType=[]):
    stimulus=[[],[]]
    dt = 1e-3
    f=0.5
    Time=10
    
    ampName=str(amp).replace('.','p')
    dataLoc='/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/'+nwType+' Results/'

    stimulus[0].append((stimulus__(biasType='AC',onAmp=amp,T=Time,dt=dt,f=f)))
    stimulus[1].append((stimulus__(biasType='Drain',T=Time,dt=dt))) #we don't want this drain to be active during training

    stimulus2 = [item[0] for item in stimulus] #go through each list in the list and find the ith item
    results_minMax=runSim(connectivity__(graph=Graph),stimulus=stimulus2, contactMode='farthest', T = Time, dt = dt, onAmp = amp, biasType='AC',f=f,junctionMode='tunneling',findFirst=False)
    results={'JunctionConductance':results_minMax.junctionConductance,'JunctionVoltage':results_minMax.junctionVoltage,'Switch':results_minMax.junctionSwitch,'Source':results_minMax.sources[0:2],'Drain':results_minMax.drains[0:2],'Signal':results_minMax.stimulus,'conductance':results_minMax.conductance}
    set_trace()
    compressed_pickle(results,dataLoc+'_NLT_Simulation_10sec'+'_Rewired'+nwType+'Networks_'+ampName+'v.pkl')
    
    nodesList= np.array([i for i in Graph.nodes])#only want all nodes

    nwSqu =[None]*len(nodesList)
    ResultSqu = nonLinearTrans_specificNodes(results_minMax,'Square',nodesList, repeats=50) #simulation, type of signal, number of nodes to sample from, number of linear regression repetitions (take avg)
    nwSqu=ResultSqu['accuracy']

    return nwSqu

In [ ]:
def get_MC_Voltsweep_Elegans(Graph=[],amp = 2,nwType=[], dt = 0.01, dur = 1, n_outs = 100, init_state = None, return_sim = False):
    ampName=str(amp).replace('.','p')
    dataLoc='/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/'+nwType+' Results/'


    signal = signal_expand(np.random.random(1000)*2*amp-amp, dur)
    results_minMax = runSim(connectivity__(graph=Graph), T = 10, dt = dt,
                start_state =  init_state,
                biasType = 'Custom', customSignal = signal, 
                junctionMode = 'tunneling', 
                contactMode = 'farthest',
                findFirst=False, disable_tqdm=True)
    
    results={'JunctionConductance':results_minMax.junctionConductance[-1],'JunctionVoltage':results_minMax.junctionVoltage[-1],'Switch':results_minMax.junctionSwitch[-1],'Source':results_minMax.sources[0:2],'Drain':results_minMax.drains[0:2],'Signal':results_minMax.stimulus,'conductance':results_minMax.conductance}
    compressed_pickle(results,dataLoc+'_MC_Simulation_'+'_'+nwType+'Networks_'+ampName+'v.pkl')
    
    n_outs=len(Graph)
    mcRAND = [MC_k(results_minMax, k, n_outs = n_outs) for k in range(dur, len(Graph)*dur, dur)]
    
    if return_sim:
        return np.sum(mcRAND), results_minMax
    else:
        return np.sum(mcRAND)

In [ ]:
resultElegans=[]
for amp in tqdm(onAmp):
    resultElegans.append(getNLT_Elegans(Graph=Elegans['G'],amp=amp,nwType='Elegans'))

In [ ]:
resultMC_Elegans=[]
for i in range(10):
    temp=[]
    
    for amp in tqdm(onAmp):
        temp.append(get_MC_Voltsweep_Elegans(Graph=Elegans['G'],amp=amp,nwType='Elegans'))
    resultMC_Elegans.append(temp)


#### Analyse Results:

In [ ]:
dfElegans=pd.DataFrame(np.mean(resultElegans,axis=1))
# dfElegansMC=pd.DataFrame(np.mean(resultMC_Elegans,axis=0))

In [ ]:
plt.rcParams['pdf.fonttype'] = 42

fig,axes=plt.subplots(1,1,figsize=(10,8))
ax=axes
ax.set_title('C Elegans Network ')
a=ax.imshow(dfElegans,cmap=plt.cm.inferno,aspect='0.5',vmin=0.55,vmax=0.85)
plt.colorbar(a,ax=ax,label='NLTavg')
ax.set_ylabel('Voltage')
ax.set_yticks(np.arange(0,len(onAmp)))
ax.set_yticklabels(onAmp)

for (j,i),label in np.ndenumerate(dfElegans):
    ax.text(i,j,np.around(label,2),ha='center',va='center',color='k')
    
plt.savefig(saveFig+'cElegans_VSweep_AvgNLT_10sec_text.pdf',format='pdf',dpi=300)

In [ ]:
plt.rcParams['pdf.fonttype'] = 42

fig,axes=plt.subplots(1,1,figsize=(10,8))
ax=axes
ax.set_title('C Elegans Network ')
a=ax.imshow(dfElegansMC,cmap=plt.cm.inferno,aspect='0.5',vmin=0,vmax=120)
plt.colorbar(a,ax=ax,label='MCavg')
ax.set_ylabel('Voltage')
ax.set_yticks(np.arange(0,len(onAmp)))
ax.set_yticklabels(onAmp)

for (j,i),label in np.ndenumerate(dfElegansMC):
    ax.text(i,j,np.around(label,2),ha='center',va='center',color='white')
    
plt.savefig(saveFig+'cElegans_VSweep_AvgMC_text.pdf',format='pdf',dpi=300)

## Create/Load Other Networks similar to C Elegans:

In [ ]:
elegansAvgDeg=np.mean(np.array([value for (node,value) in Elegans['G'].degree()])) #Avg Degree of C Elegans
elegansEdges=Elegans['G'].number_of_edges()
elegansNodes=Elegans['G'].number_of_nodes()
elegansGraph=Elegans['G']

### NWN

In [ ]:
name='/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks/VaryingDensity300NWset2_NWN.pkl'
file = open(name, 'rb')
[NWN] = pickle.load(file)
Modular=False
# name='/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/300nw_2_modules_10sets_NWN.pkl'
# NWN=decompress_pickle(name)
# Modular=True

In [ ]:
networks=[]
import glob
import os
path='/import/silo2/aloe8475/Documents/CODE/Analysis/Generate Networks/connectivity_data/'
networks=[]
files=glob.glob(path+'./*.mat')
files.sort(key=os.path.basename) #sort by creation time (name works weirdly)
for i in files:
    networks.append(loadmat(i))

In [ ]:
if Modular:
    G=NWN[1][5]['G']

In [ ]:
if not Modular:
    networks2=[]
    for i in range(len(NWN)):
        for j in range(len(NWN[i])):
            networks2.append(NWN[i][j])

In [ ]:
if not Modular:
    G2=[]
    for i in tqdm(range(len(networks2))):
        G2.append(nx.from_numpy_array(networks2[i]['adj_matrix']))
    G1=[]
    for i in range(len(networks)):
        G1.append(nx.from_numpy_array(networks[i]['adj_matrix']))

In [ ]:
G=[]
G=G1
G.append(G2[97])

In [ ]:
if Modular:
    nwnGraph=G#[97] #network with ~same avg degree as c elegans
else:
    nwnGraphs=G
nwnAvgDeg=[];nwnEdges=[];nwnNodes=[];mod=[]
for i in range(len(nwnGraphs)):
    nwnAvgDeg.append(np.mean([val for (node,val) in nwnGraphs[i].degree()]))
    nwnEdges.append(nwnGraphs[i].number_of_edges())
    nwnNodes.append(nwnGraphs[i].number_of_nodes())
    mod.append(bct.community_louvain(np.array(nx.adjacency_matrix(nwnGraphs[i]).todense()),gamma=1.1))

In [ ]:
q=np.array(mod).T[1]

In [ ]:
nwnGraph=(max(connected_component_subgraphs(nwnGraph), key=len))

In [ ]:
pos=nx.kamada_kawai_layout(nwnGraphs[1])
nx.draw(nwnGraphs[1],pos=pos)

In [ ]:
init=[]
for amp in tqdm(onAmp):
    for i in range(len(nwnGraphs)):
        init.append(inputPacker(getNLT_Elegans,Graph=nwnGraphs[i],amp=amp,nwType='NWN'))

In [ ]:
resultNWN=para_run(getNLT_Elegans,init,nCPU=10,start_method='fork')

In [ ]:
resultNWN=np.array(resultNWN).reshape(11,10)
stdresultNWN=np.std(resultNWN,axis=1)/np.sqrt(10)
avgresultNWN=np.mean(resultNWN,axis=1)

In [ ]:
plt.plot(avgresultNWN)
plt.fill_between(range(len(avgresultNWN)),avgresultNWN-stdresultNWN,avgresultNWN+stdresultNWN,alpha=0.4)

In [ ]:
initMC=[]
for amp in tqdm(onAmp):
    for i in range(len(nwnGraphs)):
        initMC.append(inputPacker(get_MC_Voltsweep_Elegans,Graph=nwnGraphs[i],amp=amp,nwType='NWN'))

In [ ]:
resultMC=[]
for i in tqdm(range(10)):
    resultMC.append(para_run(get_MC_Voltsweep_Elegans,initMC,nCPU=10,start_method='fork'))

In [ ]:
resultMC2=np.mean(resultMC,axis=0)
resultMC2std=np.std(resultMC,axis=0)/np.sqrt(10)

In [ ]:
resultMC2=np.array(resultMC2).reshape(11,10)
resultMC2std=np.array(resultMC2std).reshape(11,10)
stdresultMCNWN=np.mean(resultMC2std,axis=1)
avgresultMCNWN=np.mean(resultMC2,axis=1)

In [ ]:
plt.plot(avgresultMCNWN)
plt.fill_between(range(len(avgresultMCNWN)),avgresultMCNWN-stdresultMCNWN,avgresultMCNWN+stdresultMCNWN,alpha=0.4)

In [ ]:
dfNWN=pd.DataFrame(resultNWN)
# dfNWNMC=pd.DataFrame(resultMC2)

In [ ]:
plt.rcParams['pdf.fonttype'] = 42

fig,axes=plt.subplots(1,1,figsize=(10,8))
ax=axes
ax.set_title('NWN Network')
a=ax.imshow(dfNWN,cmap=plt.cm.inferno,aspect='auto',vmin=0.55,vmax=0.85)
plt.colorbar(a,ax=ax,label='NLTavg')
ax.set_ylabel('Voltage')
ax.set_yticks(np.arange(0,len(onAmp)))
ax.set_yticklabels(onAmp)

for (j,i),label in np.ndenumerate(dfNWN):
    ax.text(i,j,np.around(label,2),ha='center',va='center',color='k')
    
plt.savefig(saveFig+'NWN_VSweep_AvgNLT_10sec_text.pdf',format='pdf',dpi=300)

In [ ]:
plt.rcParams['pdf.fonttype'] = 42

fig,axes=plt.subplots(1,1,figsize=(10,8))
ax=axes
ax.set_title('NWN Network')
a=ax.imshow(dfNWNMC,cmap=plt.cm.inferno,aspect='auto',vmin=0,vmax=125)
plt.colorbar(a,ax=ax,label='MCavg')
ax.set_ylabel('Voltage')
ax.set_yticks(np.arange(0,len(onAmp)))
ax.set_yticklabels(onAmp)

for (j,i),label in np.ndenumerate(dfNWNMC):
    ax.text(i,j,np.around(label,2),ha='center',va='center',color='w')
    
plt.savefig(saveFig+'NWN_VSweep_AvgMC_10sec_text.pdf',format='pdf',dpi=300)

In [ ]:
def connected_component_subgraphs(G):
    for c in nx.connected_components(G):
        yield G.subgraph(c)

### Watts Strogatz

In [ ]:
numNodes=nwnGraph.number_of_nodes()
k=np.round(np.mean(np.array([i for i in nwnGraph.degree()])[:,1])) #rounded avg degree
k=np.int(k)
temp=(nx.watts_strogatz_graph(numNodes,k,p=0.17,seed=10))
WS=(max(connected_component_subgraphs(temp), key=len))


In [ ]:
resultWS=[]
for amp in tqdm(onAmp):
    resultWS.append(getNLT_Elegans(Graph=WS,amp=amp,nwType='WS'))

In [ ]:
resultMC_WS=[]
for i in range(10):
    temp=[]
    for amp in tqdm(onAmp):
        temp.append(get_MC_Voltsweep_Elegans(Graph=WS,amp=amp,nwType='WS'))
    resultMC_WS.append(temp)

In [ ]:
dfWS=pd.DataFrame(np.mean(resultWS,axis=1))
dfWSMC=pd.DataFrame(np.mean(resultMC_WS,axis=0))

In [ ]:
plt.rcParams['pdf.fonttype'] = 42

fig,axes=plt.subplots(1,1,figsize=(10,8))
ax=axes
ax.set_title('Watts Strogatz')
a=ax.imshow(list(resultWS),cmap=plt.cm.inferno,aspect='0.5',vmin=0.55,vmax=0.85)
plt.colorbar(a,ax=ax,label='NLTavg')
ax.set_ylabel('Voltage')
ax.set_yticks(np.arange(0,len(onAmp)))
ax.set_yticklabels(onAmp)

for (j,i),label in np.ndenumerate(resultWS):
    ax.text(i,j,np.around(label,2),ha='center',va='center',color='white')
    
plt.savefig(saveFig+'WattsStrogatz_NEdgesSameAsElegans_VSweep_AvgNLT_text.pdf',format='pdf',dpi=300)

In [ ]:
plt.rcParams['pdf.fonttype'] = 42

fig,axes=plt.subplots(1,1,figsize=(10,8))
ax=axes
ax.set_title('Watts Strogatz')
a=ax.imshow(dfWSMC,cmap=plt.cm.inferno,aspect='0.5',vmin=0,vmax=120)
plt.colorbar(a,ax=ax,label='MCavg')
ax.set_ylabel('Voltage')
ax.set_yticks(np.arange(0,len(onAmp)))
ax.set_yticklabels(onAmp)

for (j,i),label in np.ndenumerate(dfWSMC):
    ax.text(i,j,np.around(label,2),ha='center',va='center',color='white')
    
plt.savefig(saveFig+'WattsStrogatz_NEdgesSameAsElegans_VSweep_AvgMC_text.pdf',format='pdf',dpi=300)

### Crossbar:

In [ ]:
### If we want to Generate Crossbar:
from networkx.algorithms import bipartite

In [ ]:
# crossbar_graph=bipartite.generators.gnmk_random_graph(139,139,139**2,seed=10) #fully connected 
sparseCrossbar=[]
for i in range(10):
    sparseCrossbar.append(bipartite.generators.gnmk_random_graph(139,139,elegansEdges,seed=i)) #same edges as c elegans
# top = nx.bipartite.sets(crossbar_graph)[0]
# pos=nx.kamada_kawai_layout(crossbar_graph)

In [ ]:
crossbarAvgDeg=np.mean(np.array([value for (node,value) in crossbar_graph.degree()]))
crossbarEdges=crossbar_graph.number_of_edges()
crossbarNodes=crossbar_graph.number_of_nodes()

In [ ]:
crossbarEdges

In [ ]:
nx.draw(crossbar_graph,pos=pos, node_color='r')
nx.draw_networkx_nodes(crossbar_graph, nodelist=[102,169],pos=pos, node_color='g')

In [ ]:
resultCrossbar=[]
for amp in tqdm(onAmp):
    resultCrossbar.append(getNLT_Elegans(Graph=crossbar_graph,amp=amp,nwType='Barpartite'))

In [ ]:
resultSparseCrossbar=[]
for amp in tqdm(onAmp):
    resultSparseCrossbar.append(getNLT_Elegans(Graph=sparseCrossbar,amp=amp,nwType='Barpartite'))

In [ ]:
init=[]
for amp in tqdm(onAmp):
    for i in range(len(sparseCrossbar)):
        init.append(inputPacker(getNLT_Elegans,Graph=sparseCrossbar[i],amp=amp,nwType='Random'))

In [ ]:
resultSparseCrossbar=para_run(getNLT_Elegans,init,nCPU=10,start_method='fork')

In [ ]:
initMC=[]
for amp in tqdm(onAmp):
    for i in range(len(nwnGraphs)):
        initMC.append(inputPacker(get_MC_Voltsweep_Elegans,Graph=sparseCrossbar[i],amp=amp,nwType='Random'))

In [ ]:
resultSparseCrossbar_MC=[]
for i in tqdm(range(10)):
    resultSparseCrossbar_MC.append(para_run(get_MC_Voltsweep_Elegans,initMC,nCPU=10,start_method='fork'))

In [ ]:
resultSparseCrossbar=np.array(resultSparseCrossbar).reshape(11,10)
stdresultSparseCrossbar=np.std(resultSparseCrossbar,axis=1)/np.sqrt(10)
avgresultSparseCrossbar=np.mean(resultSparseCrossbar,axis=1)

In [ ]:
plt.plot(avgresultSparseCrossbar)
plt.fill_between(range(len(avgresultSparseCrossbar)),avgresultSparseCrossbar-stdresultSparseCrossbar,avgresultSparseCrossbar+stdresultSparseCrossbar,alpha=0.4)

In [ ]:
resultSparseCrossbar_MC2=np.mean(resultSparseCrossbar_MC,axis=0)
resultSparseCrossbar_stdMC=np.std(resultSparseCrossbar_MC,axis=0)/np.sqrt(10)

In [ ]:
resultMCSparseCrossbar=np.array(resultSparseCrossbar_MC2).reshape(11,10)
resultMCSparseCrossbarstd=np.array(resultSparseCrossbar_stdMC).reshape(11,10)
stdresultMCSparseCrossbar=np.mean(resultMCSparseCrossbarstd,axis=1)
avgresultMCSparseCrossbar=np.mean(resultMCSparseCrossbar,axis=1)

In [ ]:
resultMC_Crossbar=[]
for i in range(10):
    temp=[]
    for amp in tqdm(onAmp):
        temp.append(get_MC_Voltsweep_Elegans(Graph=crossbar_graph,amp=amp,nwType='Barpartite'))
    resultMC_Crossbar.append(temp)

In [ ]:
resultMC_SparseCrossbar=[]
for i in range(10):
    temp=[]
    for amp in tqdm(onAmp):
        temp.append(get_MC_Voltsweep_Elegans(Graph=sparseCrossbar,amp=amp,nwType='Barpartite'))
    resultMC_SparseCrossbar.append(temp)

In [ ]:
# dfCrossbar=pd.DataFrame(np.mean(resultCrossbar,axis=1))
# dfCrossbarMC=pd.DataFrame(np.mean(resultMC_Crossbar,axis=0))

dfSparseCrossbar=pd.DataFrame(np.mean(resultSparseCrossbar,axis=1))
# dfSparseCrossbarMC=pd.DataFrame(np.mean(resultMC_SparseCrossbar,axis=0))

In [ ]:
plt.rcParams['pdf.fonttype'] = 42

fig,axes=plt.subplots(1,1,figsize=(10,8))
ax=axes
ax.set_title('Crossbar Network ')
a=ax.imshow(dfSparseCrossbar,cmap=plt.cm.inferno,aspect='0.5',vmin=0.55,vmax=0.85)
plt.colorbar(a,ax=ax,label='NLTavg')
ax.set_ylabel('Voltage')
ax.set_yticks(np.arange(0,len(onAmp)))
ax.set_yticklabels(onAmp)

for (j,i),label in np.ndenumerate(dfSparseCrossbar):
    ax.text(i,j,np.around(label,2),ha='center',va='center',color='white')
    
plt.savefig(saveFig+'sparseBarpartite_NEdgesSameAsElegans_VSweep_AvgNLT_10sec_text.pdf',format='pdf',dpi=300)

In [ ]:
plt.rcParams['pdf.fonttype'] = 42

fig,axes=plt.subplots(1,1,figsize=(10,8))
ax=axes
ax.set_title('Crossbar Network')
a=ax.imshow(dfSparseCrossbarMC,cmap=plt.cm.inferno,aspect='0.5',vmin=0,vmax=120)
plt.colorbar(a,ax=ax,label='MCavg')
ax.set_ylabel('Voltage')
ax.set_yticks(np.arange(0,len(onAmp)))
ax.set_yticklabels(onAmp)

for (j,i),label in np.ndenumerate(dfSparseCrossbarMC):
    ax.text(i,j,np.around(label,2),ha='center',va='center',color='white')
    
plt.savefig(saveFig+'sparseBarpartite_NEdgesSameAsElegans_VSweep_AvgMC_text.pdf',format='pdf',dpi=300)

### ER Random

In [ ]:
# GENERATE RANDOM GRAPHS WITH SAME NUMBER OF NODES + EDGES
temp=nx.dense_gnm_random_graph(OG.number_of_nodes(),OG.number_of_edges(),seed=10)
while len(list(connected_component_subgraphs(temp))[0]) < len(OG):
    temp=nx.dense_gnm_random_graph(OG.number_of_nodes(),OG.number_of_edges())
erGraph=temp
erGraph=max(connected_component_subgraphs(erGraph), key=len)
adjMat1=np.array((nx.adjacency_matrix(erGraph)).todense())

In [ ]:
erWiredGraph=[]
adjMat2=[]
for i in tqdm(range(len(nwnGraphs))):
    OG=nwnGraphs[i].copy()
    erWiredGraph.append(nx.double_edge_swap(OG,nswap=50000,max_tries=500000,seed=10))
    erWiredGraph[i].remove_edges_from(list(nx.selfloop_edges(erWiredGraph[i])))
    erWiredGraph[i]=max(connected_component_subgraphs(erWiredGraph[i]), key=len)
    adjMat2.append(nx.adjacency_matrix(erWiredGraph[i]))

In [ ]:
# resultER=[]
# for amp in tqdm(onAmp):
#     resultER.append(getNLT_Elegans(Graph=erGraph,amp=amp,nwType='Random'))

In [ ]:
# resultERWIRED=[]
# for amp in tqdm(onAmp):
#     resultERWIRED.append(getNLT_Elegans(Graph=erWiredGraph,amp=amp,nwType='Random'))

In [ ]:
# resultMC_ER=[]
# for i in range(10):
#     temp=[]
#     for amp in tqdm(onAmp):
#         temp.append(get_MC_Voltsweep_Elegans(Graph=erGraph,amp=amp,nwType='Random'))
#     resultMC_ER.append(temp)

In [ ]:
init=[]
for amp in tqdm(onAmp):
    for i in range(len(nwnGraphs)):
        init.append(inputPacker(getNLT_Elegans,Graph=erWiredGraph[i],amp=amp,nwType='Random'))

In [ ]:
getNLT_Elegans(Graph=erWiredGraph[0],amp=amp,nwType='Random')

Running Simulation : 100%|██████████| 10000/10000 [00:25<00:00, 399.32it/s]


> <ipython-input-77-b0628621e18b>(17)getNLT_Elegans()
     15     results={'JunctionConductance':results_minMax.junctionConductance,'JunctionVoltage':results_minMax.junctionVoltage,'Switch':results_minMax.junctionSwitch,'Source':results_minMax.sources[0:2],'Drain':results_minMax.drains[0:2],'Signal':results_minMax.stimulus,'conductance':results_minMax.conductance}
     16     set_trace()
---> 17     compressed_pickle(results,dataLoc+'_NLT_Simulation_10sec'+'_Rewired'+nwType+'Networks_'+ampName+'v.pkl')
     18 
     19     nodesList= np.array([i for i in Graph.nodes])#only want all nodes

ipdb> results
{'JunctionConductance': array([[1.00000000e-07, 1.00000000e-07, 1.00000000e-07, ...,
        1.00000000e-07, 1.00000000e-07, 1.00000000e-07],
       [1.00000000e-07, 1.00000000e-07, 1.00000000e-07, ...,
        1.00000000e-07, 1.00000000e-07, 1.00000000e-07],
       [1.00000000e-07, 1.00000000e-07, 1.00000000e-07, ...,
        1.00000000e-07, 1.00000000e-07, 1.00000000e-07],
       ...,


In [ ]:
resultNWN_ERWIRED=para_run(getNLT_Elegans,init,nCPU=10,start_method='fork')

In [ ]:
resultNWN_ERWIRED=np.array(resultNWN_ERWIRED).reshape(11,10)
stdresultRewired=np.std(resultNWN_ERWIRED,axis=1)/np.sqrt(10)
avgresultRewired=np.mean(resultNWN_ERWIRED,axis=1)

In [ ]:
plt.plot(avgresultRewired)
plt.fill_between(range(len(avgresultRewired)),avgresultRewired-stdresultRewired,avgresultRewired+stdresultRewired,alpha=0.4)

In [ ]:
initMC=[]
for amp in tqdm(onAmp):
    for i in range(len(nwnGraphs)):
        initMC.append(inputPacker(get_MC_Voltsweep_Elegans,Graph=erWiredGraph[i],amp=amp,nwType='Random'))

In [ ]:
resultMC_ERWIRED=[]
for i in tqdm(range(10)):
    resultMC_ERWIRED.append(para_run(get_MC_Voltsweep_Elegans,initMC,nCPU=10,start_method='fork'))

In [ ]:
resultMCRewired=np.mean(resultMC_ERWIRED,axis=0)
resultMCRewiredStd=np.std(resultMC_ERWIRED,axis=0)/np.sqrt(10)

In [ ]:
resultMCRewiredStd=np.array(resultMCRewiredStd).reshape(11,10)
resultMCRewired=np.array(resultMCRewired).reshape(11,10)
stdresultMCRewired=np.mean(resultMCRewiredStd,axis=1)
avgresultMCRewired=np.mean(resultMCRewired,axis=1)

In [ ]:
plt.plot(avgresultMCRewired)
plt.fill_between(range(len(avgresultMCRewired)),avgresultMCRewired-stdresultMCRewired,avgresultMCRewired+stdresultMCRewired,alpha=0.4)

In [ ]:
# dfER=pd.DataFrame(np.mean(resultER,axis=1))
# dfERMC=pd.DataFrame(np.mean(resultMC_ER,axis=0))
dfERWired=pd.DataFrame(np.mean(resultERWIRED,axis=1))
# dfERWiredMC=pd.DataFrame(np.mean(resultMC_ERWIRED,axis=0))

In [ ]:
plt.rcParams['pdf.fonttype'] = 42

# fig,axes=plt.subplots(1,1,figsize=(10,8))
# ax=axes
# ax.set_title('ER Network ')
# a=ax.imshow(dfER,cmap=plt.cm.inferno,aspect='0.5',vmin=0.55,vmax=0.85)
# plt.colorbar(a,ax=ax,label='NLTavg')
# ax.set_ylabel('Voltage')
# ax.set_yticks(np.arange(0,len(onAmp)))
# ax.set_yticklabels(onAmp)

# for (j,i),label in np.ndenumerate(dfER):
#     ax.text(i,j,np.around(label,2),ha='center',va='center',color='white')
    
# plt.savefig(saveFig+'RandomER_VSweep_AvgNLT_text.pdf',format='pdf',dpi=300)

fig,axes=plt.subplots(1,1,figsize=(10,8))
ax=axes
ax.set_title('ER Rewired Network ')
a=ax.imshow(dfERWired,cmap=plt.cm.inferno,aspect='0.5',vmin=0.55,vmax=0.85)
plt.colorbar(a,ax=ax,label='NLTavg')
ax.set_ylabel('Voltage')
ax.set_yticks(np.arange(0,len(onAmp)))
ax.set_yticklabels(onAmp)

for (j,i),label in np.ndenumerate(dfERWired):
    ax.text(i,j,np.around(label,2),ha='center',va='center',color='white')
    
plt.savefig(saveFig+'RandomERWired_VSweep_AvgNLT_10sec_text.pdf',format='pdf',dpi=300)

In [ ]:
plt.rcParams['pdf.fonttype'] = 42

# fig,axes=plt.subplots(1,1,figsize=(10,8))
# ax=axes
# ax.set_title('ER Network ')
# a=ax.imshow(dfERMC,cmap=plt.cm.inferno,aspect='0.5',vmin=0,vmax=120)
# plt.colorbar(a,ax=ax,label='MCavg')
# ax.set_ylabel('Voltage')
# ax.set_yticks(np.arange(0,len(onAmp)))
# ax.set_yticklabels(onAmp)

# for (j,i),label in np.ndenumerate(dfERMC):
#     ax.text(i,j,np.around(label,2),ha='center',va='center',color='white')
    
# plt.savefig(saveFig+'randomER_VSweep_AvgMC_text.pdf',format='pdf',dpi=300)

fig,axes=plt.subplots(1,1,figsize=(10,8))
ax=axes
ax.set_title('ER Rewired Network ')
a=ax.imshow(dfERWiredMC,cmap=plt.cm.inferno,aspect='0.5',vmin=0,vmax=120)
plt.colorbar(a,ax=ax,label='MCavg')
ax.set_ylabel('Voltage')
ax.set_yticks(np.arange(0,len(onAmp)))
ax.set_yticklabels(onAmp)

for (j,i),label in np.ndenumerate(dfERWiredMC):
    ax.text(i,j,np.around(label,2),ha='center',va='center',color='white')
    
plt.savefig(saveFig+'RandomERWired_VSweep_AvgMC_text.pdf',format='pdf',dpi=300)

In [ ]:
# resultMC_Elegans=np.mean(resultMC_Elegans,axis=0)
# resultMC_NWN=np.mean(resultMC_NWN,axis=0)
# resultMC_ER=np.mean(resultMC_ER,axis=0)
# resultMC_ERWIRED=np.mean(resultMC_ERWIRED,axis=0)

In [ ]:
### SAVE RESULTS:
compressed_pickle([dfElegans,dfSparseCrossbar,dfERWired,dfNWN],saveResults+'AllNetworks_10secNLT_SameDensityAsElegans')
# compressed_pickle([dfElegansMC,dfWSMC,dfCrossbarMC,dfSparseCrossbarMC,dfERMC,dfERWiredMC,dfNWNMC],saveResults+'AllNetworksMC_SameDensityAsElegans')

In [ ]:
# [dfElegansMC,dfWSMC,dfCrossbarMC,dfSparseCrossbarMC,dfERMC,dfERWiredMC,dfNWNMC]=decompress_pickle(saveResults+'AllNetworksMC_SameDensityAsElegans')
[dfElegans,dfSparseCrossbar,dfERWired,dfNWN]=decompress_pickle(saveResults+'AllNetworks_10secNLT_SameDensityAsElegans')


## Compare Networks

In [ ]:
top = nx.bipartite.sets(crossbar_graph)[0]
top2 = nx.bipartite.sets(sparseCrossbar)[0]

In [ ]:
fig,axes=plt.subplots(3,3,figsize=(10,12))

axes[0][0].set_title('NWN')
pos=nx.kamada_kawai_layout(nwnGraph)
nx.draw(nwnGraph,pos=pos,node_size=20,ax=axes[0][0])

axes[0][1].set_title('WS')
pos=nx.kamada_kawai_layout(WS)
nx.draw(WS,pos=pos,node_size=20,ax=axes[0][1])

axes[0][2].set_title('Elegans')
pos=nx.kamada_kawai_layout(Elegans['G'])
nx.draw(Elegans['G'],pos=pos,node_size=20,ax=axes[0][2])

axes[1][0].set_title('ER')
pos=nx.kamada_kawai_layout(erGraph)
nx.draw(erGraph,pos=pos,node_size=20,ax=axes[1][0])

axes[1][1].set_title('NWN Rewired')
pos=nx.kamada_kawai_layout(erWiredGraph)
nx.draw(erWiredGraph,pos=pos,node_size=20,ax=axes[1][1])

axes[1][2].set_title('Sparse Crossbar')
pos=nx.bipartite_layout(sparseCrossbar,top2)
nx.draw(sparseCrossbar,pos=pos,node_size=20,ax=axes[1][2])

axes[2][0].set_title('Crossbar')
pos=nx.bipartite_layout(crossbar_graph,top)
nx.draw(crossbar_graph,pos=pos,node_size=20,ax=axes[2][0])


plt.savefig(saveFig+'different_networks_same_nodes+edges.pdf',format='pdf',dpi=300)

### Difference between NETWORKS:

In [ ]:
# import time
# t = time.time()
ciElegans,qElegans=bct.community_louvain(np.array(nx.adjacency_matrix(elegansGraph).todense()),gamma=1.1)
# pcoeffElegans=bct.participation_coef(np.squeeze(np.array(nx.adjacency_matrix(elegansGraph).todense())),ciElegans)
# mzElegans=bct.module_degree_zscore(np.squeeze(np.array(nx.adjacency_matrix(elegansGraph).todense())),ciElegans)
clusteringElegans=nx.clustering(elegansGraph)
diamElegans=nx.diameter(elegansGraph)
# degElegans=[val for (nodes,val) in elegansGraph.degree()]
# avgDegElegans=np.mean(degElegans)
# SPLElegans=nx.shortest_path_length(Elegans['G'],source=getFarthestPairing(Elegans['adj_matrix'])[0],target=getFarthestPairing(Elegans['adj_matrix'])[1])
# avgSPLElegans=nx.average_shortest_path_length(elegansGraph)
# localEfficiencyElegans=nx.local_efficiency(elegansGraph)
# globalEfficiencyElegans=nx.global_efficiency(elegansGraph)

print('Elegans Finished')
# elapsed = time.time() - t
# print(elapsed)
qNWN=np.zeros(len(nwnGraphs));clusteringNWN=[];diamNWN=[]
for i in range(len(nwnGraphs)):
    ciNWN,qNWN[i]=bct.community_louvain(np.array(nx.adjacency_matrix(nwnGraphs[i]).todense()),gamma=1.1)
#     pcoeffNWN=bct.participation_coef(np.squeeze(np.array(nx.adjacency_matrix(nwnGraph).todense())),ciNWN)
#     mzNWN=bct.module_degree_zscore(np.squeeze(np.array(nx.adjacency_matrix(nwnGraph).todense())),ciNWN)
    clusteringNWN.append(nx.clustering(nwnGraphs[i]))
    diamNWN.append(nx.diameter(nwnGraphs[i]))

# degNWN=[val for (nodes,val) in nwnGraph.degree()]
# avgDegNWN=np.mean(degNWN)
# SPLNWN=nx.shortest_path_length(nwnGraph,source=getFarthestPairing(nx.adjacency_matrix(nwnGraph).todense())[0],target=getFarthestPairing(nx.adjacency_matrix(nwnGraph).todense())[1])
# avgSPLNWN=nx.average_shortest_path_length(nwnGraph)
# localEfficiencyNWN=nx.local_efficiency(nwnGraph)
# globalEfficiencyNWN=nx.global_efficiency(nwnGraph)
print('NWN Finished')

# ciER,qER=bct.community_louvain(np.array(nx.adjacency_matrix(erGraph).todense()),gamma=1.1)
# pcoeffER=bct.participation_coef(np.squeeze(np.array(nx.adjacency_matrix(erGraph).todense())),ciER)
# mzER=bct.module_degree_zscore(np.squeeze(np.array(nx.adjacency_matrix(erGraph).todense())),ciER)
# clusteringER=nx.clustering(erGraph)
# degER=[val for (nodes,val) in erGraph.degree()]
# avgDegER=np.mean(degER)
# SPLER=nx.shortest_path_length(erGraph,source=getFarthestPairing(nx.adjacency_matrix(erGraph).todense())[0],target=getFarthestPairing(nx.adjacency_matrix(erGraph).todense())[1])
# avgSPLER=nx.average_shortest_path_length(erGraph)
# localEfficiencyER=nx.local_efficiency(erGraph)
# globalEfficiencyER=nx.global_efficiency(erGraph)
# print('ER Finished')
qERWired=np.zeros(len(nwnGraphs));clusteringERWired=[];diamERWired=[]
for i in range(len(nwnGraphs)):
    ciERWired,qERWired[i]=bct.community_louvain(np.array(nx.adjacency_matrix(erWiredGraph[i]).todense()),gamma=1.1)
    # pcoeffERWired=bct.participation_coef(np.squeeze(np.array(nx.adjacency_matrix(erWiredGraph).todense())),ciERWired)
    # mzERWired=bct.module_degree_zscore(np.squeeze(np.array(nx.adjacency_matrix(erWiredGraph).todense())),ciERWired)
    clusteringERWired.append(nx.clustering(erWiredGraph[i]))
    diamERWired.append(nx.diameter(erWiredGraph[i]))
                   
# degERWired=[val for (nodes,val) in erWiredGraph.degree()]
# avgDegERWired=np.mean(degERWired)
# SPLERWired=nx.shortest_path_length(erWiredGraph,source=getFarthestPairing(nx.adjacency_matrix(erWiredGraph).todense())[0],target=getFarthestPairing(nx.adjacency_matrix(erWiredGraph).todense())[1])
# avgSPLERWired=nx.average_shortest_path_length(erWiredGraph)
# localEfficiencyERWired=nx.local_efficiency(erWiredGraph)
# globalEfficiencyERWired=nx.global_efficiency(erWiredGraph)
print('Rewired Finished')


# ciCrossbar,qCrossbar=bct.community_louvain(np.array(nx.adjacency_matrix(crossbar_graph).todense()),gamma=1.1)
# pcoeffCrossbar=bct.participation_coef(np.squeeze(np.array(nx.adjacency_matrix(crossbar_graph).todense())),ciCrossbar)
# mzCrossbar=bct.module_degree_zscore(np.squeeze(np.array(nx.adjacency_matrix(crossbar_graph).todense())),ciCrossbar)
# clusteringCrossbar=nx.clustering(crossbar_graph)
# degCrossbar=[val for (nodes,val) in crossbar_graph.degree()]
# avgDegCrossbar=np.mean(degCrossbar)
# SPLCrossbar=nx.shortest_path_length(crossbar_graph,source=getFarthestPairing(nx.adjacency_matrix(crossbar_graph).todense())[0],target=getFarthestPairing(nx.adjacency_matrix(crossbar_graph).todense())[1])
# avgSPLCrossbar=nx.average_shortest_path_length(crossbar_graph)
# localEfficiencyCrossbar=nx.local_efficiency(crossbar_graph)
# globalEfficiencyCrossbar=nx.global_efficiency(crossbar_graph)
# print('Crossbar Finished')
qSparseCrossbar=np.zeros(len(nwnGraphs));clusteringSparseCrossbar=[];diamSparseCrossbar=[]
for i in range(len(nwnGraphs)):
    ciSparseCrossbar,qSparseCrossbar[i]=bct.community_louvain(np.array(nx.adjacency_matrix(sparseCrossbar[i]).todense()),gamma=1.1)
# pcoeffSparseCrossbar=bct.participation_coef(np.squeeze(np.array(nx.adjacency_matrix(sparseCrossbar).todense())),ciSparseCrossbar)
# mzSparseCrossbar=bct.module_degree_zscore(np.squeeze(np.array(nx.adjacency_matrix(sparseCrossbar).todense())),ciSparseCrossbar)
    clusteringSparseCrossbar.append(nx.clustering(sparseCrossbar[i]))
    diamSparseCrossbar.append(nx.diameter(sparseCrossbar[i]))
# degSparseCrossbar=[val for (nodes,val) in sparseCrossbar.degree()]
# avgDegSparseCrossbar=np.mean(degSparseCrossbar)
# SPLSparseCrossbar=nx.shortest_path_length(sparseCrossbar,source=getFarthestPairing(nx.adjacency_matrix(sparseCrossbar).todense())[0],target=getFarthestPairing(nx.adjacency_matrix(sparseCrossbar).todense())[1])
# avgSPLSparseCrossbar=nx.average_shortest_path_length(sparseCrossbar)
# localEfficiencySparseCrossbar=nx.local_efficiency(sparseCrossbar)
# globalEfficiencySparseCrossbar=nx.global_efficiency(sparseCrossbar)
print('Sparse Crossbar Finished')


# ciWS,qWS=bct.community_louvain(np.array(nx.adjacency_matrix(WS).todense()),gamma=1.1)
# pcoeffWS=bct.participation_coef(np.squeeze(np.array(nx.adjacency_matrix(WS).todense())),ciWS)
# mzWS=bct.module_degree_zscore(np.squeeze(np.array(nx.adjacency_matrix(WS).todense())),ciWS)
# clusteringWS=nx.clustering(WS)
# degWS=[val for (nodes,val) in WS.degree()]
# avgDegWS=np.mean(degWS)
# SPLWS=nx.shortest_path_length(WS,source=getFarthestPairing(nx.adjacency_matrix(WS).todense())[0],target=getFarthestPairing(nx.adjacency_matrix(WS).todense())[1])
# avgSPLWS=nx.average_shortest_path_length(WS)
# localEfficiencyWS=nx.local_efficiency(WS)
# globalEfficiencyWS=nx.global_efficiency(WS)
# print('WS Finished')


In [ ]:
clstrMeanNWN=[];clstrMeanSparseCross=[];clstrMeanRewired=[];
for i in range(len(clusteringNWN)):
    clstrMeanNWN.append(np.mean(list(clusteringNWN[i].values())))
    clstrMeanSparseCross.append(np.mean(list(clusteringSparseCrossbar[i].values())))
    clstrMeanRewired.append(np.mean(list(clusteringERWired[i].values())))

In [ ]:
np.mean(tmp)

#### SMALL WORLDNESS

In [ ]:
nwnAdjMat=[];rewiredAdjMat=[];sparseCrossAdjMat=[]
for i in range(len(nwnGraphs)):
    nwnAdjMat.append(np.array(nx.adjacency_matrix(nwnGraphs[i]).todense()))
    rewiredAdjMat.append(np.array(nx.adjacency_matrix(erWiredGraph[i]).todense()))
    sparseCrossAdjMat.append(np.array(nx.adjacency_matrix(sparseCrossbar[i]).todense()))

In [ ]:
AdjMat={'AdjMatElegans':np.array(nx.adjacency_matrix(elegansGraph).todense()),'AdjMatNWN':nwnAdjMat,'AdjMatRewired':rewiredAdjMat,'AdjMatSparseCrossbar':sparseCrossAdjMat}

In [ ]:
savemat(saveLoc+'AllNetworks_AdjMats',AdjMat)

In [ ]:
temp=loadmat('/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/WS Networks/AllNetworks_AdjMats_smallworld')
temp2=loadmat('/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/Network Comparison/AllNetworks_AdjMats_smallworld')
smallworld=temp2['smallworld']
# smallworldNWN=temp['smallworld'][0][0]
# smallworldWS=temp['smallworldWS'][0][0]
smallworldElegans=temp['smallworldElegans'][0][0]
# smallworldCrossbar=temp['smallworldCrossbar'][0][0]
# smallworldSparseCrossbar=temp['smallworldSparseCrossbar'][0][0]
# smallworldER=temp['smallworldER'][0][0]
# smallworldRewired=temp['smallworldRewired'][0][0]

In [ ]:
fig,axes=plt.subplots(3,3,figsize=(15,10))
ax=axes[1][0]
# ax.bar(1,np.mean(list(clusteringCrossbar.values())),label='Crossbar',color='green')
ax.bar(1,np.mean(clstrMeanSparseCross),yerr=np.std(clstrMeanSparseCross),label='Sparse Crossbar',color='blue',capsize=10)
# ax.bar(3,np.mean(list(clusteringER.values())),label='ER Random',color='blue')
ax.bar(2,np.mean(clstrMeanRewired),yerr=np.std(clstrMeanRewired),label='Random Network',color='green',capsize=10)
ax.bar(3,np.mean(list(clusteringElegans.values())),label='Elegans',color='orange',capsize=10)
# ax.bar(6,np.mean(list(clusteringWS.values())),label='WS',color='k')
ax.bar(4,np.mean(clstrMeanNWN),yerr=np.std(clstrMeanNWN),label='NWN',color='red',capsize=10)
ax.set_xticks([1,2,3,4])
ax.set_ylabel('Average Clustering')
ax.set_xticklabels([])
ax.set_xticklabels(['Crossbar','Sparse Crossbar','ER','NWN Rewired','Elegans','WS','NWN'],rotation=45)

ax=axes[0][0]
# ax.bar(1,np.mean(list(clusteringCrossbar.values())),label='Crossbar',color='green')
ax.bar(1,np.mean(diamSparseCrossbar),yerr=np.std(diamSparseCrossbar),label='Sparse Crossbar',color='blue',capsize=10)
# ax.bar(3,np.mean(list(clusteringER.values())),label='ER Random',color='blue')
ax.bar(2,np.mean(diamERWired),yerr=np.std(diamERWired),label='Random Network',color='green',capsize=10)
ax.bar(3,diamElegans,label='Elegans',color='orange',capsize=10)
# ax.bar(6,np.mean(list(clusteringWS.values())),label='WS',color='k')
ax.bar(4,np.mean(diamNWN),yerr=np.std(diamNWN),label='NWN',color='red',capsize=10)
ax.set_xticks([1,2,3,4])
ax.set_ylabel('Diameter')
ax.set_xticklabels([])

ax=axes[0][1]
# ax.bar(1,np.mean(list(clusteringCrossbar.values())),label='Crossbar',color='green')
ax.bar(1,np.mean(qSparseCrossbar),yerr=np.std(qSparseCrossbar),label='Sparse Crossbar',color='blue',capsize=10)
# ax.bar(3,np.mean(list(clusteringER.values())),label='ER Random',color='blue')
ax.bar(2,np.mean(qERWired),yerr=np.std(qERWired),label='Random Network',color='green',capsize=10)
ax.bar(3,qElegans,label='Elegans',color='orange',capsize=10)
# ax.bar(6,np.mean(list(clusteringWS.values())),label='WS',color='k')
ax.bar(4,np.mean(qNWN),yerr=np.std(qNWN),label='NWN',color='red',capsize=10)
ax.set_xticks([1,2,3,4])
ax.set_ylabel('Q')
ax.set_xticklabels([])

ax=axes[1][1]
# ax.bar(1,np.mean(list(clusteringCrossbar.values())),label='Crossbar',color='green')
ax.bar(1,np.mean(smallworld[2]),yerr=np.std(smallworld[2]),label='Sparse Crossbar',color='blue',capsize=10)
# ax.bar(3,np.mean(list(clusteringER.values())),label='ER Random',color='blue')
ax.bar(2,np.mean(smallworld[1]),yerr=np.std(smallworld[1]),label='Random Network',color='green',capsize=10)
ax.bar(3,smallworldElegans,label='Elegans',color='orange',capsize=10)
# ax.bar(6,np.mean(list(clusteringWS.values())),label='WS',color='k')
ax.bar(4,np.mean(smallworld[0]),yerr=np.std(smallworld[0]),label='NWN',color='red',capsize=10)
ax.set_xticks([1,2,3,4])
ax.set_ylabel('SWP')
ax.set_xticklabels([])

ax.set_xticklabels(['Crossbar','Sparse Crossbar','ER','NWN Rewired','Elegans','WS','NWN'],rotation=45)
plt.savefig(saveFig+'GraphTheory+Elegans_NWN_ER_Crossbar_10Networks.pdf',format='pdf',dpi=300)

In [ ]:
smallworldRewired

In [ ]:
dfSparseCrossbar

In [ ]:
# Crossbar_dict={'graph':crossbar_graph,'SW':smallworldCrossbar,'Mod':qCrossbar,'pcoeff':pcoeffCrossbar,'mz':mzCrossbar,'clustering':clusteringCrossbar,'degree':degCrossbar,'avgDeg':avgDegCrossbar,'localEfficiency':localEfficiencyCrossbar,'globalEfficiency':globalEfficiencyCrossbar,'NLT':dfCrossbar,'MC':dfCrossbarMC}
SparseCrossbar_dict={'graph':sparseCrossbar,'SW':smallworldSparseCrossbar,'Mod':qSparseCrossbar,'pcoeff':pcoeffSparseCrossbar,'mz':mzSparseCrossbar,'clustering':clusteringSparseCrossbar,'degree':degSparseCrossbar,'avgDeg':avgDegSparseCrossbar,'localEfficiency':localEfficiencySparseCrossbar,'globalEfficiency':globalEfficiencySparseCrossbar,'NLT':dfSparseCrossbar,'MC':dfSparseCrossbarMC}
Elegans_dict={'graph':elegansGraph,'SW':smallworldElegans,'Mod':qElegans,'pcoeff':pcoeffElegans,'mz':mzElegans,'clustering':clusteringElegans,'degree':degElegans,'avgDeg':avgDegElegans,'localEfficiency':localEfficiencyElegans,'globalEfficiency':globalEfficiencyElegans,'NLT':dfElegans,'MC':dfElegansMC}
NWN_dict={'graph':nwnGraph,'SW':smallworldNWN,'Mod':qNWN,'pcoeff':pcoeffNWN,'mz':mzNWN,'clustering':clusteringNWN,'degree':degNWN,'avgDeg':avgDegNWN,'localEfficiency':localEfficiencyNWN,'globalEfficiency':globalEfficiencyNWN,'NLT':dfNWN,'MC':dfNWNMC}
# WS_dict={'graph':WS,'SW':smallworldWS,'Mod':qWS,'pcoeff':pcoeffWS,'mz':mzWS,'clustering':clusteringWS,'degree':degWS,'avgDeg':avgDegWS,'localEfficiency':localEfficiencyWS,'globalEfficiency':globalEfficiencyWS,'NLT':dfWS,'MC':dfWSMC}
# ER_dict={'graph':erGraph,'SW':smallworldER,'Mod':qER,'pcoeff':pcoeffER,'mz':mzER,'clustering':clusteringER,'degree':degER,'avgDeg':avgDegER,'localEfficiency':localEfficiencyER,'globalEfficiency':globalEfficiencyER,'NLT':dfER,'MC':dfERMC}
Rewired_dict={'graph':erWiredGraph,'SW':smallworldRewired,'Mod':qERWired,'pcoeff':pcoeffERWired,'mz':mzERWired,'clustering':clusteringERWired,'degree':degERWired,'avgDeg':avgDegERWired,'localEfficiency':localEfficiencyERWired,'globalEfficiency':globalEfficiencyERWired,'NLT':dfERWired,'MC':dfERWiredMC}

compressed_pickle([SparseCrossbar_dict,Elegans_dict,NWN_dict,Rewired_dict],saveResults+'NetworksComparisonDicts')

In [ ]:
# Rewired_dict['NLT']=pd.DataFrame(resultERWIRED)
# Rewired_dict['MC']=pd.DataFrame(resultMC_ERWIRED)

# compressed_pickle([Crossbar_dict,SparseCrossbar_dict,Elegans_dict,NWN_dict,ER_dict,Rewired_dict,WS_dict],saveResults+'NetworksComparisonDicts')

In [ ]:
#UNPACK RESULTS
[SparseCrossbar_dict,Elegans_dict,NWN_dict,Rewired_dict]=decompress_pickle(saveResults+'NetworksComparisonDicts')
resultElegans=Elegans_dict['NLT'];resultNWN=NWN_dict['NLT'];resultERWIRED=Rewired_dict['NLT'];resultSparseCrossbar=SparseCrossbar_dict['NLT']
resultMC_Elegans=Elegans_dict['MC'];resultMC_NWN=NWN_dict['MC'];resultMC_ERWIRED=Rewired_dict['MC'];resultMC_SparseCrossbar=SparseCrossbar_dict['MC']
elegansGraph=Elegans_dict['graph'];nwnGraph=NWN_dict['graph'];erWiredGraph=Rewired_dict['graph'];sparseCrossbar=SparseCrossbar_dict['graph']

In [ ]:
stdresultMCSparseCrossbar

In [ ]:
fig,axes=plt.subplots(1,2,figsize=(10,4))
ax=axes[0]
ax.plot(avgresultNWN,'-o',label='NWN',color='red')
ax.fill_between(range(len(avgresultNWN)),avgresultNWN-stdresultNWN,avgresultNWN+stdresultNWN,alpha=0.4,color='red')
ax.plot(avgresultRewired,'-o',label='Random Network',color='green')
ax.fill_between(range(len(avgresultRewired)),avgresultRewired-stdresultRewired,avgresultRewired+stdresultRewired,alpha=0.4,color='green')
ax.plot(avgresultSparseCrossbar,'-o',label='Sparse Crossbar',color='blue')
ax.fill_between(range(len(avgresultSparseCrossbar)),avgresultSparseCrossbar-stdresultSparseCrossbar,avgresultSparseCrossbar+stdresultSparseCrossbar,alpha=0.4,color='blue')

ax.plot(resultElegans,'-o',label='Elegans',color='orange')

# ax.plot(resultWS,'-o',label='WS',color='k')

ax.set_xlabel('Voltage')
ax.set_xticks(np.arange(0,len(onAmp)))
ax.set_xticklabels(onAmp)
ax.set_ylabel('NLT')
ax.set_ylim([0.55,0.85])
# ax.legend()

ax=axes[1]
# ax.plot(np.mean(resultMC_Crossbar,axis=0),'-o',label='Crossbar',color='green')
ax.plot(avgresultMCNWN,'-o',label='NWN',color='red')
ax.fill_between(range(len(avgresultMCNWN)),avgresultMCNWN-stdresultMCNWN,avgresultMCNWN+stdresultMCNWN,alpha=0.4,color='red')
ax.plot(avgresultMCRewired,'-o',label='Random Network',color='green')
ax.fill_between(range(len(avgresultMCRewired)),avgresultMCRewired-stdresultMCRewired,avgresultMCRewired+stdresultMCRewired,alpha=0.4,color='green')
ax.plot(avgresultMCSparseCrossbar,'-o',label='Sparse Crossbar',color='blue')
ax.fill_between(range(len(avgresultMCSparseCrossbar)),avgresultMCSparseCrossbar-stdresultMCSparseCrossbar,avgresultMCSparseCrossbar+stdresultMCSparseCrossbar,alpha=0.4,color='blue')

ax.plot(resultMC_Elegans,'-o',label='Elegans',color='orange')

# ax.plot(np.mean(resultMC_ER,axis=0),'-o',label='ER',color='blue')
# ax.plot(np.mean(resultMC_WS,axis=0),'-o',label='WS',color='k')

ax.set_xlabel('Voltage')
ax.set_ylabel('MC')
ax.set_xticks(np.arange(0,len(onAmp)))
ax.set_xticklabels(onAmp)
ax.legend()

# plt.savefig(saveFig+'NetworkComparison_Elegans_NWN_ER_Crossbar_NLT+MC_10sec_300Nodes_2000Edges.pdf',format='pdf',dpi=300)

In [ ]:
resultsNLT=np.array([np.array(resultElegans),np.array(resultSparseCrossbar),np.array(resultERWIRED),np.array(resultNWN)])
resultsMC=np.array([np.array(resultMC_Elegans),np.array(resultMC_SparseCrossbar),np.array(resultMC_ERWIRED),np.array(resultMC_NWN)])

In [ ]:
## NORMALIZED RESULTS:

    
maxMCval=np.max(np.max(resultsMC))
maxNLTval=np.max(np.max(resultsNLT))
minMCval=np.min(np.min(resultsMC))
minNLTval=np.min(np.min(resultsNLT))

dfMC_diffMax=[]
dfNLT_diffMax=[]

for j in range(len(onAmp)):
    for i in range(len(resultsNLT)):
        dfMC_diffMax.append(1-((maxMCval-resultsMC[i][j])/(maxMCval-minMCval)))
    for i in range(len(resultsMC)):
        dfNLT_diffMax.append(1-((maxNLTval-resultsNLT[i][j])/(maxNLTval-minNLTval)))


In [ ]:
plt.plot(np.array(dfNLT_diffMax).reshape(11,4))

In [ ]:
fig,axes=plt.subplots(3,3,figsize=(15,10))
# ax=axes[0][0]
# # ax.bar(1,SPLCrossbar,label='Crossbar',color='green')
# ax.bar(2,SPLSparseCrossbar,label='Sparse Crossbar',color='tab:green')
# # ax.bar(3,SPLER,label='ER Random',color='blue')
# ax.bar(4,SPLERWired,label='ER Rewired Random',color='tab:red')
# ax.bar(5,SPLElegans,label='Elegans',color='orange')
# # ax.bar(6,SPLWS,label='WS',color='k')
# ax.bar(7,SPLNWN,label='NWN',color='red')

# ax.set_xticks([1,2,3,4,5,6,7])
# ax.set_ylabel('Shortest Path (Source to Drain)')
# ax.set_xticklabels([])

ax=axes[0][0]
# ax.bar(1,np.mean(list(clusteringCrossbar.values())),label='Crossbar',color='green')
ax.bar(2,np.mean(list(clusteringSparseCrossbar.values())),label='Sparse Crossbar',color='tab:green')
# ax.bar(3,np.mean(list(clusteringER.values())),label='ER Random',color='blue')
ax.bar(4,np.mean(list(clusteringERWired.values())),label='ER Rewired Random',color='tab:red')
ax.bar(5,np.mean(list(clusteringElegans.values())),label='Elegans',color='orange')
# ax.bar(6,np.mean(list(clusteringWS.values())),label='WS',color='k')
ax.bar(7,np.mean(list(clusteringNWN.values())),label='NWN',color='red')
ax.set_xticks([1,2,3,4,5,6,7])
ax.set_ylabel('Global Clustering')
ax.set_xticklabels([])


ax=axes[2][0]
# ax.bar(1,np.mean(avgSPLCrossbar),label='Crossbar',color='green')
ax.bar(2,np.mean(avgSPLSparseCrossbar),label='Sparse Crossbar',color='tab:green')
# ax.bar(3,np.mean(avgSPLER),label='ER Random',color='blue')
ax.bar(4,np.mean(avgSPLERWired),label='ER Rewired Random',color='tab:red')
ax.bar(5,np.mean(avgSPLElegans),label='Elegans',color='orange')
# ax.bar(6,np.mean(avgSPLWS),label='WS',color='k')
ax.bar(7,np.mean(avgSPLNWN),label='NWN',color='red')
ax.set_xticks([1,2,3,4,5,6,7])
ax.set_ylabel('Avg SPL')

ax.set_xticklabels(['Crossbar','Sparse Crossbar','ER','NWN Rewired','Elegans','WS','NWN'],rotation=45)

ax=axes[0][1]
# ax.bar(1,np.mean(Crossbar_dict['Mod']),label='Crossbar',color='green')
ax.bar(2,np.mean(SparseCrossbar_dict['Mod']),label='Sparse Crossbar',color='tab:green')
# ax.bar(3,np.mean(ER_dict['Mod']),label='ER Random',color='blue')
ax.bar(4,np.mean(Rewired_dict['Mod']),label='ER Rewired Random',color='tab:red')
ax.bar(5,np.mean(Elegans_dict['Mod']),label='Elegans',color='orange')
# ax.bar(6,np.mean(WS_dict['Mod']),label='WS',color='k')
ax.bar(7,np.mean(NWN_dict['Mod']),label='NWN',color='red')
ax.set_xticks([1,2,3,4,5,6,7])
ax.set_ylim(0,0.75)
ax.set_ylabel('Modularity (q)')


ax=axes[1][1]
# ax.bar(1,np.mean(Crossbar_dict['SW']),label='Crossbar',color='green')
ax.bar(2,np.mean(SparseCrossbar_dict['SW']),label='Sparse Crossbar',color='tab:green')
# ax.bar(3,np.mean(ER_dict['SW']),label='ER Random',color='blue')
ax.bar(4,np.mean(Rewired_dict['SW']),label='ER Rewired Random',color='tab:red')
ax.bar(5,np.mean(Elegans_dict['SW']),label='Elegans',color='orange')
# ax.bar(6,np.mean(WS_dict['SW']),label='WS',color='k')
ax.bar(7,np.mean(NWN_dict['SW']),label='NWN',color='red')
ax.set_xticks([1,2,3,4,5,6,7])
ax.set_ylabel('Small World Propensity')

ax.set_xticklabels(['Crossbar','Sparse Crossbar','ER','NWN Rewired','Elegans','WS','NWN'],rotation=45)


ax=axes[0][2]
ax.plot(resultElegans,'-o',label='Elegans',color='orange')
ax.plot(resultNWN,'-o',label='NWN',color='red')
ax.plot(resultERWIRED,'-o',label='NWN Rewired',color='tab:red')
# ax.plot(resultER,'-o',label='ER',color='blue')
# ax.plot(resultCrossbar,'-o',label='Crossbar',color='green')
ax.plot(resultSparseCrossbar,'-o',label='Sparse Crossbar',color='tab:green')
# ax.plot(resultWS,'-o',label='WS',color='k')

ax.set_xlabel('Voltage')
ax.set_xticks(np.arange(0,len(onAmp)))
ax.set_xticklabels(onAmp)
ax.set_ylabel('NLT')
ax.set_ylim([0.55,0.85])

ax=axes[1][2]
# ax.plot(np.mean(resultMC_Crossbar,axis=0),'-o',label='Crossbar',color='green')
ax.plot(resultMC_SparseCrossbar,'-o',label='Sparse Crossbar',color='tab:green')

ax.plot(dfElegansMC,'-o',label='Elegans',color='orange')
ax.plot(dfNWNMC,'-o',label='NWN',color='red')
ax.plot(resultMC_ERWIRED,'-o',label='NWN Rewired',color='tab:red')
# ax.plot(np.mean(resultMC_ER,axis=0),'-o',label='ER',color='blue')
# ax.plot(np.mean(resultMC_WS,axis=0),'-o',label='WS',color='k')

ax.set_xlabel('Voltage')
ax.set_ylabel('MC')
ax.set_xticks(np.arange(0,len(onAmp)))
ax.set_xticklabels(onAmp)
ax.legend()

axes[2][1].axis('off')
axes[2][2].axis('off')

plt.savefig(saveFig+'GraphTheory+NetworkComparison_Elegans_NWN_ER_Crossbar_NLT+MC_10sec_300Nodes_2000Edges_AvgSPL.pdf',format='pdf',dpi=300)

In [ ]:
onAmp[3]/SPLCrossbar

In [ ]:
fig,axes=plt.subplots(1,1,figsize=(5,5))
ax=axes
ax.plot(1,(resultCrossbar[7]),'o',label='Crossbar',color='green')
ax.plot(2,(resultSparseCrossbar[7]),'o',label='Sparse Crossbar',color='tab:green')
ax.plot(3,(resultER[7]),'o',label='ER Random',color='blue')
ax.plot(4,(resultERWIRED[7]),'o',label='ER Rewired Random',color='tab:blue')
ax.plot(5,(resultElegans[7]),'o',label='Elegans',color='orange')
ax.plot(6,(resultWS[7]),'o',label='WS',color='k')
ax.plot(7,(resultNWN[7]),'o',label='NWN',color='red')

ax.plot(1,(resultCrossbar[3]),'o',label='Crossbar',color='green',alpha=0.5)
ax.plot(2,(resultSparseCrossbar[3]),'o',label='Sparse Crossbar',color='tab:green',alpha=0.5)
ax.plot(3,(resultER[3]),'o',label='ER Random',color='blue',alpha=0.5)
ax.plot(4,(resultERWIRED[3]),'o',label='ER Rewired Random',color='tab:blue',alpha=0.5)
ax.plot(5,(resultElegans[3]),'o',label='Elegans',color='orange',alpha=0.5)
ax.plot(6,(resultWS[3]),'o',label='WS',color='k',alpha=0.5)
ax.plot(7,(resultNWN[3]),'o',label='NWN',color='red',alpha=0.5)

ax.plot(1,(resultCrossbar[1]),'o',label='Crossbar',color='green',alpha=0.2)
ax.plot(2,(resultSparseCrossbar[1]),'o',label='Sparse Crossbar',color='tab:green',alpha=0.2)
ax.plot(3,(resultER[1]),'o',label='ER Random',color='blue',alpha=0.2)
ax.plot(4,(resultERWIRED[1]),'o',label='ER Rewired Random',color='tab:blue',alpha=0.2)
ax.plot(5,(resultElegans[1]),'o',label='Elegans',color='orange',alpha=0.2)
ax.plot(6,(resultWS[1]),'o',label='WS',color='k',alpha=0.2)
ax.plot(7,(resultNWN[1]),'o',label='NWN',color='red',alpha=0.2)

# ax.set_ylim([0.55,0.85])
ax.set_xticks([1,2,3,4,5,6,7])
ax.set_ylabel('NLT')
ax.set_xticklabels(['Crossbar','Sparse Crossbar','ER Random','ER Rewired Random','Elegans','WS','NWN'],rotation=45)

In [ ]:
resultNWN

In [ ]:
[avgSPLCrossbar,avgSPLSparseCrossbar,avgSPLER,avgSPLERWired,avgSPLElegans,avgSPLWS,avgSPLNWN]

In [ ]:
[SPLCrossbar,SPLSparseCrossbar,SPLER,SPLERWired,SPLElegans,SPLWS,SPLNWN]

In [ ]:
fig,axes=plt.subplots(3,4,figsize=(10,8),sharex=True,sharey=True)
i=0
fig.subplots_adjust(hspace=0.6)
fig.subplots_adjust(wspace=0.3)
for ax in tqdm(axes.flat):
    if i < 11:
        ax.set_title(str(onAmp[i]) + 'V') 
        ax.plot([avgSPLCrossbar,avgSPLSparseCrossbar,avgSPLER,avgSPLERWired,avgSPLElegans,avgSPLWS,avgSPLNWN],np.hstack([resultCrossbar[i],resultSparseCrossbar[i],resultER[i],resultERWIRED[i],resultElegans[i],resultWS[i],resultNWN[i]]),'o')#,vmin=0.55,vmax=0.85)
        ax.set_xlabel('Average SPL')
        ax.set_ylim([0.55,0.90])
        ax.set_ylabel('NLT')
#         ax.set_xlim([1,7])
        ax.tick_params(axis='x', colors='tab:blue')
        ax.xaxis.label.set_color('tab:blue')

        ax2=ax.twiny()
        ax2.plot([SPLCrossbar,SPLSparseCrossbar,SPLER,SPLERWired,SPLElegans,SPLWS,SPLNWN],[resultCrossbar[i],resultSparseCrossbar[i],resultER[i],resultERWIRED[i],resultElegans[i],resultWS[i],resultNWN[i]],'o',c='r')#,vmin=0.55,vmax=0.85)
        ax2.set_xlabel('SPL (Source to Drain)')
        ax2.set_ylim([0.55,0.90])
#         ax2.set_xlim([3,15])
        ax2.tick_params(axis='x', colors='red')
        ax2.xaxis.label.set_color('red')

    else:
        ax.axis('off')
    i=i+1
# plt.savefig(saveLoc+'Avg SPL & Source-Drain SPL vs NLT - VSweep 300NWN Varying Modularity.pdf',format='pdf',dpi=300)

### FUNCTIONAL CONNECTIVITY

In [ ]:
voltNames=['0p2','0p5','0p75','1','1p25','1p5','1p75','2','3','5','10']
nwNames4Load=['Elegans','SparseBarpartite','RewiredRandom','NWN']
# graphs=[crossbar_graph,sparseCrossbar,erGraph,erWiredGraph,elegansGraph,WS,nwnGraph]
nwNames4=['Elegans','SparseCrossbar','NWNRewired','NWN']
graphs4=[elegansGraph,sparseCrossbar,erWiredGraph,nwnGraph]
results=[resultElegans,resultMC_SparseCrossbar,resultERWIRED,resultNWN]
resultsMC=[resultMC_Elegans,resultMC_SparseCrossbar,resultMC_ERWIRED,resultMC_NWN]

In [ ]:
# dataLocNames=['Barpartite','Barpartite','Random','Random','Elegans','WS','NWN']
dataLocNames4=['Elegans','Barpartite','Random','NWN']

# dataLoc=[]
# for i in range(len(dataLocNames)):
#     dataLoc.append('/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/'+dataLocNames[i]+' Results/')

dataLoc4=[]
for i in range(len(dataLocNames4)):
    dataLoc4.append('/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/'+dataLocNames4[i]+' Results/')

# dataLocNWN='/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/NWN Results/'
# dataLocWS='/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/WS Results/'
# dataLocElegans='/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/Elegans Results/'

In [ ]:
NLT_sims=[[] for i in range(len(nwNames4Load))]
MC_sims=[[] for i in range(len(nwNames4Load))]
for i in range(len(nwNames4Load)):
    for volt in tqdm(range(len(resultNWN)-1)):
        name=dataLoc4[i]+'_NLT_Simulation_10sec'+'_'+nwNames4Load[i]+'Networks_'+voltNames[volt]+'v.pkl'
#         name2=dataLoc4[i]+'_MC_Simulation_'+'_'+nwNames4Load[i]+'Networks_'+voltNames[volt]+'v.pkl'
        NLT_sims[i].append(decompress_pickle(name))
        MC_sims[i].append(decompress_pickle(name2))
        
# NWN_NLT_sims=[]
# NWN_MC_sims=[]
# for volt in tqdm(range(len(resultNWN))):
#     name=dataLocNWN+'_NLT_Simulation_'+'_NWNNetworks_'+voltNames[volt]+'v.pkl'
#     name2=dataLocNWN+'_MC_Simulation_'+'_NWNNetworks_'+voltNames[volt]+'v.pkl'
#     NWN_NLT_sims.append(decompress_pickle(name))
#     NWN_MC_sims.append(decompress_pickle(name2))

In [ ]:
#Subgraph AdjMat

#Threshold by conductance - when tunnelling becomes appreciable (offResistance * 10)

# def getOnGraph(sims,network,edgeList,numWires):#, this_TimeStamp = 0):
# #     edgeList = network['edge_list']
#     adjMat = np.zeros((numWires, numWires))
# #     set_trace()
#     adjMat[edgeList[:,0], edgeList[:,1]] = (1/sims['JunctionResistance'][-1])>1e-06#network.junctionSwitch[this_TimeStamp,:] #CHANGE THIS TO CONDUCTANCE THRESHOLD?
#     adjMat[edgeList[:,1], edgeList[:,0]] = (1/sims['JunctionResistance'][-1])>1e-06#network.junctionSwitch[this_TimeStamp,:] #CHANGE THIS TO CONDUCTANCE THRESHOLD?
#     onGraph = nx.from_numpy_array(adjMat)
#     onGraph=nx.DiGraph.to_undirected(onGraph)
    
#     return onGraph

def getWeightedGraph(sims,network,edgeList,numWires):#, this_TimeStamp = 0):
#     edgeList = network['edge_list']
    adjMat = np.zeros((numWires, numWires))
#     set_trace()
    adjMat[edgeList[:,0], edgeList[:,1]] = sims['JunctionConductance'][-1]#network.junctionSwitch[this_TimeStamp,:] #CHANGE THIS TO CONDUCTANCE THRESHOLD?
    adjMat[edgeList[:,1], edgeList[:,0]] = sims['JunctionConductance'][-1]#network.junctionSwitch[this_TimeStamp,:] #CHANGE THIS TO CONDUCTANCE THRESHOLD?
    WeightedGraph = nx.from_numpy_array(adjMat)
    WeightedGraph=nx.DiGraph.to_undirected(WeightedGraph)
    
    return WeightedGraph

In [ ]:
sources=[[] for i in range(len(nwNames4))]
drains=[[] for i in range(len(nwNames4))]
sourcesMC=[[] for i in range(len(nwNames4))]
drainsMC=[[] for i in range(len(nwNames4))]
weightedSubGraph_NLT=[[] for i in range(len(nwNames4))]
weightedSubGraph_MC=[[] for i in range(len(nwNames4))]
for i in tqdm(range(len(nwNames4))):
    for j in range(len(resultNWN)):
            sim=NLT_sims[i][j]
            graph=graphs4[i]
            network=np.array(nx.adjacency_matrix(graph).todense())

            sources[i].append(sim['Source'])
            drains[i].append(sim['Drain'])
            OGgraph=graph
            pos=nx.kamada_kawai_layout(OGgraph)
            numWires=graph.number_of_nodes()
            sim['JunctionResistance']=1/sim['JunctionConductance'][-1]
            edgeList=np.array(list(graph.edges()))
    #         subGraph=getOnGraph(sim,network,edgeList,numWires)
            weightedSubGraph_NLT[i].append(getWeightedGraph(sim,network,edgeList,numWires))
    for j in range(len(resultNWN)):
            sim=MC_sims[i][j]
            graph=graphs4[i]
            network=np.array(nx.adjacency_matrix(graph).todense())

            sourcesMC[i].append(sim['Source'])
            drainsMC[i].append(sim['Drain'])
            OGgraph=graph
            pos=nx.kamada_kawai_layout(OGgraph)
            numWires=graph.number_of_nodes()
            sim['JunctionResistance']=1/sim['JunctionConductance'][-1]
            edgeList=np.array(list(graph.edges()))
    #         subGraph=getOnGraph(sim,network,edgeList,numWires)
            weightedSubGraph_MC[i].append(getWeightedGraph(sim,network,edgeList,numWires))

In [ ]:
plt.figure(figsize=(20,3))
plt.plot(sim['Signal'][0].signal,'-o')
plt.xlim([0,200])

In [ ]:
compressed_pickle([weightedSubGraph_MC,weightedSubGraph_NLT],saveLoc+'WeightedGraphsAllNWS_10sec')

In [ ]:
[weightedSubGraph_MC,weightedSubGraph_NLT]=decompress_pickle(saveLoc+'WeightedGraphsAllNWS_10sec')

In [ ]:
# sourcesMC=[]
# drainsMC=[]
# weightedSubGraphNWN_MC=[]
# for k in tqdm(range(len(resultNWN))):
#         sim=NWN_MC_sims[k]
#         graph=nwnGraph
#         network=np.array(nx.adjacency_matrix(graph).todense())
    
#         sourcesMC.append(sim['Source'])
#         drainsMC.append(sim['Drain'])
#         OGgraph=graph
#         pos=nx.kamada_kawai_layout(OGgraph)
#         numWires=graph.number_of_nodes()
# #         sim['JunctionResistance']=1/sim['JunctionConductance']
#         edgeList=np.array(list(graph.edges()))
# #         subGraph=getOnGraph(sim,network,edgeList,numWires)
#         weightedSubGraphNWN_MC.append(getWeightedGraph(sim,network,edgeList,numWires))

In [ ]:
resultMC_Elegans2=np.mean(resultMC_Elegans,axis=0)

In [ ]:
resultMC_WS2=np.mean(resultMC_WS,axis=0)

In [ ]:
resultMC_NWN2=np.mean(resultMC_NWN,axis=0)

In [ ]:
len(weightedSubGraph_NLT[0])

In [ ]:
maxWeightsNLT=[]
minWeightsNLT=[]
maxWeightsMC=[]
minWeightsMC=[]
for i in range(len(weightedSubGraph_NLT)):
    weights=[]
    weightsMin=[]
    for j in range(len(weightedSubGraph_NLT[i])):
        maxNW=np.argmax(results[i][0][j])
        minNW=np.argmin(results[i][0][j])
        G=weightedSubGraph_NLT[i][j]
        edges=G.edges()
        weights1=[G[u][v]['weight'] for u,v in edges]
        weights.append([i * 1e6 for i in (weights1)])
    maxWeightsNLT.append(np.max(np.hstack(weights)))
    minWeightsNLT.append(np.min(np.hstack(weights)))
    
    weightsMC=[]
    weights1MC=[]
    for j in range(len(weightedSubGraph_MC[i])):
        maxNW=np.argmax(resultsMC[i][0][j])
        minNW=np.argmin(resultsMC[i][0][j])
        G=weightedSubGraph_MC[i][j]
        edges=G.edges()
        weights1MC.append([G[u][v]['weight'] for u,v in edges])
        weightsMC.append([i * 1e6 for i in (weights1MC[j])])
    if weightsMC[0]:
        maxWeightsMC.append(np.max(np.array(weightsMC)))
        minWeightsMC.append(np.min(np.array(weightsMC)))

# minWeightsNLT=np.min(minWeightsNLT)
# maxWeightsNLT=np.max(maxWeightsNLT)

In [ ]:
minWeights=np.min(np.hstack(weights1))
maxWeights=np.max(np.hstack(weights1))

In [ ]:
import matplotlib.colors as clrs

In [ ]:
#### PLOT SUBGRAPHS + CURRENT OVERLAY
#Max NLT
fig, axes = plt.subplots(11,len(results),figsize=(15,30))
#For each voltage,
for i in tqdm(range(len(weightedSubGraph_NLT))):
    for j in range(len(weightedSubGraph_NLT[i])):

            #find top + bottom network
        ax=axes[j][i]
        G=weightedSubGraph_NLT[i][j]#current_new[i][j][maxNW]
        edge_weights=nx.get_edge_attributes(G,'weight')
        G.remove_edges_from((e for e, w in edge_weights.items() if w <1e-6)) #why did we choose 1e-6
        edges=G.edges()
    #         set_trace()
        weights=[G[u][v]['weight'] for u,v in edges]
#         weights=[w * 1e6 for w in (weights)]
#         cNorm=clrs.SymLogNorm(10,vmin=minWeightsNLT,vmax=maxWeightsNLT)
#         scalarMap = plt.cm.ScalarMappable(norm=cNorm, cmap=plt.cm.inferno)
#         colorList=[]
#         for k in range(len(weights)):
#             colorVal = scalarMap.to_rgba(weights[k])
#             colorList.append(colorVal)

        OGgraph=graphs4[i]
        #draw OG graph
        if i==1:
            top = nx.bipartite.sets(OGgraph)[0]
            pos=nx.bipartite_layout(OGgraph,top)
        else:
            pos=nx.kamada_kawai_layout(OGgraph)
        h=nx.draw_networkx_nodes(OGgraph,pos=pos,node_color='grey',node_size=10,ax=ax)
        h.set_zorder(1)

        if i == 0:
            ax.set_ylabel(str(onAmp[j])+'V')
        if j == 0:
            ax.set_title(nwNames4[i])
        h2=nx.draw_networkx_edges(G,pos=pos,ax=ax,edge_color=weights,edge_cmap=plt.cm.inferno,edge_vmin=minWeights,edge_vmax=maxWeights)
    
        if h2:
            h2.set_norm(clrs.SymLogNorm(10))
            h2.set_zorder(3)
#             if j == 10 and i == 6:
#                 plt.colorbar(h2)
        nx.draw_networkx_nodes(G,pos=pos,nodelist=[sources[i][j][0]],node_color='#3f9b0b',node_size=120,node_shape ='*',ax=ax)
        nx.draw_networkx_nodes(G,pos=pos,nodelist=[drains[i][j][0]],node_color='#7e1e9c',node_size=120,node_shape ='*',ax=ax)
        ax.set_frame_on(False)
           
        ax.annotate('NLT = '+"{0:.2f}".format(results[i][0][j]),xy=(0.5,1),xycoords='axes fraction')
        ax.set_yticks([])
        ax.set_xticks([])

    plt.savefig(saveFig+'VSweep 300NWN NetworkComparison_Elegans_NWN_ER_Crossbar - Current + Threshold Weighted Graphs 10sec_NLT.pdf',format='pdf',dpi=300)

In [ ]:
#### PLOT SUBGRAPHS + CURRENT OVERLAY
#Max NLT
fig, axes = plt.subplots(11,len(results),figsize=(15,30))
#For each voltage,
for i in tqdm(range(len(weightedSubGraph_MC))):
    for j in range(len(weightedSubGraph_MC[i])):

            #find top + bottom network
        ax=axes[j][i]
        G=weightedSubGraph_MC[i][j]#current_new[i][j][maxNW]
        edge_weights=nx.get_edge_attributes(G,'weight')
        G.remove_edges_from((e for e, w in edge_weights.items() if w <1e-6))
        edges=G.edges()
    #         set_trace()
        weights=[G[u][v]['weight'] for u,v in edges]
#         weights=[w * 1e6 for w in (weights)]

#         cNorm=clrs.SymLogNorm(10,vmin=minWeightsNLT,vmax=maxWeightsNLT)
#         scalarMap = plt.cm.ScalarMappable(norm=cNorm, cmap=plt.cm.inferno)
#         colorList=[]
#         for k in range(len(weights)):
#             colorVal = scalarMap.to_rgba(weights[k])
#             colorList.append(colorVal)

        OGgraph=graphs[i]
        #draw OG graph
        if i<2:
            top = nx.bipartite.sets(OGgraph)[0]
            pos=nx.bipartite_layout(OGgraph,top)
        else:
            pos=nx.kamada_kawai_layout(OGgraph)
        h=nx.draw_networkx_nodes(OGgraph,pos=pos,node_color='grey',node_size=10,ax=ax)
        h.set_zorder(1)

        if i == 0:
            ax.set_ylabel(str(onAmp[j])+'V')
        if j == 0:
            ax.set_title(nwNames[i])
        h2=nx.draw_networkx_edges(G,pos=pos,ax=ax,edge_color=weights,edge_cmap=plt.cm.inferno,edge_vmin=np.min(np.array(weights1MC)),edge_vmax=np.max(np.array(weights1MC)))
        if h2:
            h2.set_norm(clrs.SymLogNorm(10))
            h2.set_zorder(3)
        nx.draw_networkx_nodes(G,pos=pos,nodelist=[sourcesMC[i][j][0]],node_color='#3f9b0b',node_size=120,node_shape ='*',ax=ax)
        nx.draw_networkx_nodes(G,pos=pos,nodelist=[drainsMC[i][j][0]],node_color='#7e1e9c',node_size=120,node_shape ='*',ax=ax)
        ax.set_frame_on(False)
        ax.annotate('MC = '+"{0:.2f}".format(resultsMC[i][0][j]),xy=(0.5,1),xycoords='axes fraction')
        ax.set_yticks([])
        ax.set_xticks([])


    plt.savefig(saveFig+'VSweep 300NWN NetworkComparison_Elegans_NWN_ER_Crossbar - Current + Threshold Weighted Graphs MC.pdf',format='pdf',dpi=300)

In [ ]:
bins=np.histogram(np.hstack([degWS,degElegans,degNWN]), bins=40)[1] #get the bin edges

plt.hist(degWS,bins=bins)
plt.hist(degNWN,bins=bins,alpha=0.7)
plt.hist(degElegans,bins=bins,alpha=0.7)


### CONDUCTANCE

In [ ]:
# fig,axes=plt.subplots(11,7,figsize=(15,30),sharex=True,sharey=True)
# fig.subplots_adjust(hspace=.5)
# fig.subplots_adjust(wspace=.3)

# for i in tqdm(range(len(NLT_sims))):
#     for j in range(len(NLT_sims[i])):
        
#         ax=axes[j][i]
#         if j == 0:
#             ax.set_title(nwNames[i])
#         b=ax.semilogy(NLT_sims[i][j]['conductance'],c='tab:red')
#         ax.text(0,7e-4,'NLT = '+"{0:.2f}".format(results[i][0][j]))
#         ax.set_ylim([1e-8,1e-2])
#         ax.set_ylabel('G')
#         ax.set_xlabel('t')

# plt.savefig(saveFig+'VSweep 300NWN Different Networks Conductance - NLT.pdf',format='pdf',dpi=300)

fig,axes=plt.subplots(11,7,figsize=(15,30),sharex=True,sharey=True)
fig.subplots_adjust(hspace=.5)
fig.subplots_adjust(wspace=.3)

for i in tqdm(range(len(MC_sims))):
    for j in range(len(MC_sims[i])):
        
        ax=axes[j][i]
        if j == 0:
            ax.set_title(nwNames[i])
        b=ax.semilogy(MC_sims[i][j]['conductance'],c='tab:red')
        ax.text(0,7e-4,'MC = '+"{0:.2f}".format(resultsMC[i][0][j]))
        ax.set_ylim([1e-8,1e-2])
        ax.set_ylabel('G')
        ax.set_xlabel('t')

plt.savefig(saveFig+'VSweep 300NWN Different Networks Conductance - MC.pdf',format='pdf',dpi=300)

### MULTITASK:

In [ ]:
electrodes={'Elegans':[],'SparseCrossbar':[],'NWN':[],'NWNRewired':[]}
electrodesNoise={'Elegans':[],'SparseCrossbar':[],'NWN':[],'NWNRewired':[]}

In [ ]:
electrodes['Elegans']=[225,253,7,44]#[253,190,34,44]#,145]
electrodes['SparseCrossbar']=[84,114,111,260]#[59,8,233,260]#,164]
#Normal NWN
# electrodes['NWN']=[138,194,48,140]#,253]#[165,97,252,140]#
#Modular NWN
electrodes['NWN']=[25,22,295,288]# Electrodes in each module #[25,167,132,249] # Electrodes on opposite ends
electrodes['NWNRewired']=[168,22,6,149]#[183,185,23,59]#,103] #uncomment for extra noise

In [ ]:
electrodesNoise['Elegans']=[253,190,34,44,276]
electrodesNoise['SparseCrossbar']=[59,8,233,260,41]
electrodesNoise['NWN']=[138,194,48,140,97]
electrodesNoise['NWNRewired']=[183,185,23,59,165] #uncomment for extra noise

In [ ]:
pos=nx.kamada_kawai_layout(elegansGraph)
nx.draw(elegansGraph,pos=pos,with_labels=True,node_color='grey')
nx.draw_networkx_nodes(elegansGraph,pos=pos,nodelist=[electrodes['Elegans'][0],electrodes['Elegans'][2]],node_color='green')
nx.draw_networkx_nodes(elegansGraph,pos=pos,nodelist=[electrodes['Elegans'][1],electrodes['Elegans'][3]],node_color='red')

In [ ]:
pos=nx.kamada_kawai_layout(nwnGraph)
nx.draw(nwnGraph,pos=pos,with_labels=True,node_color='grey')
nx.draw_networkx_nodes(nwnGraph,pos=pos,nodelist=[electrodes['NWN'][0],electrodes['NWN'][2]],node_color='green')
nx.draw_networkx_nodes(nwnGraph,pos=pos,nodelist=[electrodes['NWN'][1],electrodes['NWN'][3]],node_color='red')

In [ ]:
pos=nx.kamada_kawai_layout(erWiredGraph)
nx.draw(erWiredGraph,pos=pos,with_labels=True,node_color='grey')
nx.draw_networkx_nodes(erWiredGraph,pos=pos,nodelist=[electrodes['NWNRewired'][0],electrodes['NWNRewired'][2]],node_color='green')
nx.draw_networkx_nodes(erWiredGraph,pos=pos,nodelist=[electrodes['NWNRewired'][1],electrodes['NWNRewired'][3]],node_color='red')

In [ ]:
pos=nx.kamada_kawai_layout(sparseCrossbar)
nx.draw(sparseCrossbar,pos=pos,with_labels=True,node_color='grey')
nx.draw_networkx_nodes(sparseCrossbar,pos=pos,nodelist=[electrodes['SparseCrossbar'][0],electrodes['SparseCrossbar'][2]],node_color='green')
nx.draw_networkx_nodes(sparseCrossbar,pos=pos,nodelist=[electrodes['SparseCrossbar'][1],electrodes['SparseCrossbar'][3]],node_color='red')

In [ ]:
from random import gauss
from pandas import Series

In [ ]:
def MC_k_new(network, k = 1, node_list=[], transient = 100):
#     node_list=n_outs#len(node_list)
    signal = network.stimulus[0].signal[transient:]
    wire_voltage = network.wireVoltage[transient:]
    T, N = wire_voltage.shape
#     node_list = np.random.choice(range(0, N), n_outs, replace = False)
    
    lhs = wire_voltage[k:,node_list]
    rhs = signal[:-k]
    try:
        weight = np.linalg.lstsq(lhs, rhs, rcond = None)[0]
        result = np.matmul(lhs, weight)
        mc = (np.cov(rhs, result)[0,1])**2/(np.var(signal[k:])*np.var(result))
        return mc
    except:
        return 0

In [ ]:
def nonLinearTrans_specificNodes(network, targetType, nodesList, repeats = 1):
    """
    targetType can be DoubleFrequency and all other possible biasType.
    wireVoltage[T,N] can also be the first positional parameter.
    """
    from edamame.core.simSettings import stimulus__

    if targetType == 'DoubleFrequency':
        target = stimulus__(network.stimulus[0].biasType, network.TimeVector, f = 2*network.stimulus[0].f).signal
    else:
        target = stimulus__(targetType, network.TimeVector, f = network.stimulus[0].f).signal
        
    wireVoltage = network.wireVoltage    
    T, N = wireVoltage.shape
    output = dict()
    accuracy = np.zeros(repeats)
    weight = np.zeros((repeats, len(nodesList)+1))
    result = np.zeros((repeats, T))
    
    for i in range(repeats):
        lhs = np.hstack((np.ones((T, 1)), wireVoltage[:,nodesList]))
        weight[i,:] = np.linalg.lstsq(lhs, target, rcond = None)[0]
        result[i,:] = np.dot(lhs, weight[i,:])
        accuracy[i] = 1-getRNMSE(result[i,:], target)
        
    output['weight'] = np.mean(weight, axis = 0)
    output['result'] = np.mean(result, axis = 0)
    output['accuracy'] = np.mean(accuracy)
    output['target'] = target
    
    return output 

In [ ]:
def get_MC_and_NLT(Graph,j,elecs,noiseOn=False,amp = 2, dt = 0.01, dur = 1, n_outs = 100, init_state = None, return_sim = False):
    ampName=str(amp).replace('.','p')
    f=0.5

    stimulus=[]
    stimulus.append(stimulus__(biasType='AC',onAmp=amp,TimeVector=np.arange(0,10,dt),f=f))
    stimulus.append(stimulus__(biasType='Drain',TimeVector=np.arange(0,10,dt))) #we don't want this drain to be active during training
    stimulus.append(stimulus__(biasType='Custom',TimeVector=np.arange(0,10,dt),customSignal=signal_expand(np.random.random(1000)*2*amp-amp, dur)))
    stimulus.append(stimulus__(biasType='Drain',TimeVector=np.arange(0,10,dt))) #we don't want this drain to be active during training
    #White noise
    if noiseOn:
        noise=[gauss(0.0, 1.0) for i in range(1000)]
        stimulus.append(stimulus__(biasType='Custom',TimeVector=np.arange(0,10,dt),customSignal=signal_expand(noise, dur)))
    
    simRAND = runSim(connectivity__(graph=Graph), T = 10, dt = dt,
                start_state =  init_state,
                stimulus=stimulus,
                onAmp=amp,
                junctionMode = 'tunneling', 
                contactMode = 'preSet',
                electrodes=elecs,
                findFirst=False, disable_tqdm=True)

    output={'JunctionConductance':simRAND.junctionConductance[-1],'JunctionVoltage':simRAND.junctionVoltage[-1],'Switch':simRAND.junctionSwitch[-1],'Source':simRAND.sources[0:2],'Drain':simRAND.drains[0:2],'Signal':simRAND.stimulus}
    if noiseOn:
        compressed_pickle(output,dataLoc4[j]+'NLT+MC+NOISE_Electrodes_'+'NetworkComparison_'+str(j+1)+'_'+ampName+'v.pkl')
    else:
        compressed_pickle(output,dataLoc4[j]+'NLT+MC_ElectrodesParallel_'+'NetworkComparison_'+str(j+1)+'_'+ampName+'v.pkl')

   
    nodesList = []

    nodesList = np.array([i for i in Graph.nodes])
    mcRAND = [MC_k_new(simRAND, l, node_list=nodesList) for l in range(dur, len(nodesList)*dur, dur)]
    
        
    nwSqu =[None]*len(nodesList)

    countK=0

    ResultSqu = nonLinearTrans_specificNodes(simRAND,'Square',nodesList, repeats=50) #simulation, type of signal, number of nodes to sample from, number of linear regression repetitions (take avg)
    nwSqu=ResultSqu['accuracy']
    
    if return_sim:
        return np.sum(mcRAND), simRAND, nwSqu
    else:
        return np.sum(mcRAND),nwSqu

In [ ]:
electrodes

In [ ]:
nx.shortest_path_length(erWiredGraph,1,50)

In [ ]:
nx.shortest_path_length(erWiredGraph,electrodes['NWNRewired'][2],electrodes['NWNRewired'][3])

In [ ]:
noiseOn=False #Add white noise?

if noiseOn:
    elecs=electrodesNoise
else:
    elecs=electrodes
initMCNLT=[]
for i in tqdm(range(len(onAmp))): #for each voltage
    for j in range(len(graphs4)): #for each network type
        initMCNLT.append(inputPacker(get_MC_and_NLT,graphs4[j],j,elecs=elecs[nwNames4[j]],noiseOn=noiseOn,amp=onAmp[i]))

In [ ]:
get_MC_and_NLT(graphs4[3],3,elecs=elecs[nwNames4[3]],noiseOn=noiseOn,amp=onAmp[4])

In [ ]:
resultMCNLT=[None]*10
for i in tqdm(range(10)):
    resultMCNLT[i]=para_run(get_MC_and_NLT,initMCNLT,nCPU=10,start_method='fork')

In [ ]:
MC=[]
NLT=[]
for i in range(len(resultMCNLT)):
    MC.append(np.array(resultMCNLT[i]).T[0])
    NLT.append(np.hstack(np.array(resultMCNLT[i]).T[1]))

In [ ]:
MC=np.mean(MC,axis=0)
NLT=np.mean(NLT,axis=0)

In [ ]:
np.array(np.double(NLT)).reshape(11,4)

In [ ]:
MC=np.array(np.double(MC)).reshape(11,4)
NLT=np.array(np.double(NLT)).reshape(11,4)

In [ ]:
cm1 = clrs.LinearSegmentedColormap.from_list("MyCmapName",["y","b","r","g"])

In [ ]:
plt.rcParams['pdf.fonttype'] = 42

fig,axes=plt.subplots(1,2,figsize=(10,4))
fig.subplots_adjust(wspace=0.3)
ax=axes[0]
ax.set_title('NLT')
a=ax.imshow(NLT,aspect='auto',cmap=plt.cm.inferno,vmin=0.55,vmax=0.8)
ax.set_ylabel('V')
ax.set_yticks(np.arange(0,11))
ax.set_yticklabels(onAmp)
ax.set_xticks([0,1,2,3])
ax.set_xticklabels(nwNames4[0:],rotation=45)
plt.colorbar(a,ax=ax,label='NLT')
for (j,i),label in np.ndenumerate(NLT):
    ax.text(i,j,np.around(label,2),ha='center',va='center',color='k')

ax=axes[1]
ax.set_title('MC')
b=ax.imshow(MC,aspect='auto',cmap=plt.cm.inferno,vmin=0,vmax=250)
ax.set_ylabel('V')
ax.set_yticks(np.arange(0,11))
ax.set_yticklabels(onAmp)
ax.set_xticks([0,1,2,3])
ax.set_xticklabels(nwNames4[0:],rotation=45)
plt.colorbar(b,ax=ax,label='MC')
for (j,i),label in np.ndenumerate(MC):
    ax.text(i,j,np.around(label,2),ha='center',va='center',color='k')
    
# plt.savefig(saveFig+'Avg NLT+MC Same Sim allNodes - VSweep 300NWN Varying Modularity_2Modules.pdf',format='pdf',dpi=300)

In [ ]:
plt.figure(figsize=(10,8))
plt.scatter(NLT.T[0],(MC.T[0]),c='orange')
plt.scatter((NLT.T[1]),(MC.T[1]),c='b')
plt.scatter((NLT.T[2]),(MC.T[2]),c='r')
plt.scatter((NLT.T[3]),(MC.T[3]),c='g')
plt.xlabel('NLT')
plt.ylabel('MC')
plt.legend(nwNames4)

plt.figure(figsize=(10,8))
plt.scatter(np.max(NLT.T[0]),np.max(MC.T[0]),c='orange')
plt.scatter(np.max(NLT.T[1]),np.max(MC.T[1]),c='b')
plt.scatter(np.max(NLT.T[2]),np.max(MC.T[2]),c='r')
plt.scatter(np.max(NLT.T[3]),np.max(MC.T[3]),c='g')
plt.xlabel('NLT')
plt.ylabel('MC')
plt.legend(nwNames4)

In [ ]:
maxMCval=np.max(MC)
maxNLTval=np.max(NLT)

In [ ]:
#Calculate the difference between each network and the max possible performance

dfMC_diffMax=[]
for i in range(len(MC)):
    dfMC_diffMax.append(maxMCval-MC[i])
    
dfNLT_diffMax=[]
for i in range(len(NLT)):
    dfNLT_diffMax.append(maxNLTval-NLT[i])


In [ ]:
def normalize(column):
    upper = column.max()
    lower = column.min()
    y = (column - lower)/(upper-lower)
    return y

In [ ]:
# a=normalize(pd.DataFrame(np.array(dfNLT_sumDiff)))
# b=normalize(pd.DataFrame(np.array(dfMC_sumDiff)))
a=normalize(np.array(dfNLT_diffMax))
b=normalize(np.array(dfMC_diffMax))

fig,axes=plt.subplots(1,2,figsize=(10,4))
ax=axes[0]
a1=ax.imshow(b,aspect='auto',cmap=plt.cm.viridis_r)
cb=plt.colorbar(a1,ax=ax)
cb.ax.invert_yaxis() 
ax.set_title('Normalized MC')
ax.set_xticks([0,1,2,3])
ax.set_xticklabels(nwNames4[0:],rotation=45)

ax.set_ylabel('V')
ax.set_yticks(range(0,11))
ax.set_yticklabels(onAmp)
cb.set_ticks(np.around(np.arange(0,1,step=0.2),1))
cb.set_ticklabels(np.around(np.arange(0,1.1,0.2),1))



for (j,i),label in np.ndenumerate(b):
    ax.text(i,j,np.around(label,2),ha='center',va='center',color='k')


ax=axes[1]
b1=ax.imshow(a,aspect='auto',cmap=plt.cm.viridis_r)
cb=plt.colorbar(b1,ax=ax)
cb.ax.invert_yaxis() 

ax.set_title('Normalized NLT')
ax.set_xticks([0,1,2,3])
ax.set_xticklabels(nwNames4[0:],rotation=45)
ax.set_ylabel('V')
ax.set_yticks(range(0,11))
ax.set_yticklabels(onAmp)
cb.set_ticks(np.around(np.arange(0,1,step=0.2),1))
cb.set_ticklabels(np.around(np.arange(0,1.1,0.2),1))


for (j,i),label in np.ndenumerate(a):
    ax.text(i,j,np.around(label,2),ha='center',va='center',color='k')

plt.rcParams['pdf.fonttype'] = 42
# plt.savefig(saveFig+'Normalized NLT and MC Max Diff allNodes - VSweep 300NWN Varying Modularity_2Modules.pdf',format='pdf',dpi=300)


In [ ]:
normalizedDifferenceNLT_MC=(a+b)/2#/(a.T+b.T)
# normalizedDifferenceMC_NLT=(b-a)#/(b.T+a.T)

In [ ]:
import matplotlib.colors
fig,ax=plt.subplots(1)
plt.imshow(np.abs(normalizedDifferenceNLT_MC),aspect='auto',cmap=plt.cm.viridis_r)
cb=plt.colorbar(label='Max Diff (NLT - MC) ',ax=ax)
cb.ax.invert_yaxis() 
cb.set_ticks(np.arange(0,1,step=0.2))
cb.set_ticklabels(np.arange(0,1.1,0.2))


ax=plt.gca()
ax.set_xticks(np.arange(0,4))
# ax.set_xticklabels(['Integrated','1','2','3','4','Segregated','Separated'],rotation=45)
ax.set_yticks(np.arange(0,11))
ax.set_yticklabels(onAmp)
ax.set_xticks([0,1,2,3])
ax.set_xticklabels(nwNames4[0:],rotation=45)
ax.set_ylabel('V')
for (j,i),label in np.ndenumerate(np.abs(normalizedDifferenceNLT_MC)):
    ax.text(i,j,np.around(label,2),ha='center',va='center',color='k')
    

plt.rcParams['pdf.fonttype'] = 42
# plt.savefig(saveFig+'Normalized NLT + MC Max Diff allNodes - VSweep 300NWN Varying Modularity_2Modules.pdf',format='pdf',dpi=300,optimize=True)

### Subgraph/Functional Analysis

In [ ]:
MultiTask_sims=[[] for i in range(len(nwNames4))]
for i in range(len(nwNames4)):
    for volt in tqdm(range(len(NLT))):
        name2=dataLoc4[i]+'NLT+MC_ElectrodesParallel_'+'NetworkComparison_'+str(i+1)+'_'+voltNames[volt]+'v.pkl'
        MultiTask_sims[i].append(decompress_pickle(name2))

In [ ]:
#Subgraph AdjMat

#Threshold by conductance - when tunnelling becomes appreciable (offResistance * 10)

# def getOnGraph(sims,network,edgeList,numWires):#, this_TimeStamp = 0):
# #     edgeList = network['edge_list']
#     adjMat = np.zeros((numWires, numWires))
# #     set_trace()
#     adjMat[edgeList[:,0], edgeList[:,1]] = (1/sims['JunctionResistance'])>1e-06#network.junctionSwitch[this_TimeStamp,:] #CHANGE THIS TO CONDUCTANCE THRESHOLD?
#     adjMat[edgeList[:,1], edgeList[:,0]] = (1/sims['JunctionResistance'])>1e-06#network.junctionSwitch[this_TimeStamp,:] #CHANGE THIS TO CONDUCTANCE THRESHOLD?
#     onGraph = nx.from_numpy_array(adjMat)
#     onGraph=nx.DiGraph.to_undirected(onGraph)
    
#     return onGraph

def getWeightedGraph(sims,network,edgeList,numWires):#, this_TimeStamp = 0):
#     edgeList = network['edge_list']
    adjMat = np.zeros((numWires, numWires))
#     set_trace()
    adjMat[edgeList[:,0], edgeList[:,1]] = sims['JunctionConductance']#network.junctionSwitch[this_TimeStamp,:] #CHANGE THIS TO CONDUCTANCE THRESHOLD?
    adjMat[edgeList[:,1], edgeList[:,0]] = sims['JunctionConductance']#network.junctionSwitch[this_TimeStamp,:] #CHANGE THIS TO CONDUCTANCE THRESHOLD?
    WeightedGraph = nx.from_numpy_array(adjMat)
    WeightedGraph=nx.DiGraph.to_undirected(WeightedGraph)
    
    return WeightedGraph

In [ ]:
networks4=[]
for i in range(len(graphs4)):
    networks4.append(np.array(nx.adjacency_matrix(graphs4[i]).todense()))

In [ ]:
simulations4=np.array(MultiTask_sims).reshape(11,4)

In [ ]:
len(nwNames4[0])

In [ ]:
sources=[[] for i in range(len(nwNames4))]
drains=[[] for i in range(len(nwNames4))]
sourcesMC=[[] for i in range(len(nwNames4))]
drainsMC=[[] for i in range(len(nwNames4))]
weightedSubGraph=[[] for i in range(len(nwNames4))]
for i in tqdm(range(len(nwNames4))):
    for j in range(len(NLT)):
            sim=MultiTask_sims[i][j]
            graph=graphs4[i]
            network=np.array(nx.adjacency_matrix(graph).todense())

            sources[i].append(sim['Source'])
            drains[i].append(sim['Drain'])
            OGgraph=graph
            pos=nx.kamada_kawai_layout(OGgraph)
            numWires=graph.number_of_nodes()
            sim['JunctionResistance']=1/sim['JunctionConductance']
            edgeList=np.array(list(graph.edges()))
    #         subGraph=getOnGraph(sim,network,edgeList,numWires)
            weightedSubGraph[i].append(getWeightedGraph(sim,network,edgeList,numWires))

In [ ]:
maxWeightsNLT=[]
minWeightsNLT=[]
maxWeightsMC=[]
minWeightsMC=[]
for i in range(len(weightedSubGraph)):
    weights=[]
    weightsMin=[]
    for j in range(len(weightedSubGraph[i])):
        maxNW=np.argmax(NLT[j][i])
        minNW=np.argmin(NLT[j][i])
        G=weightedSubGraph[i][j]
        edges=G.edges()
        weights1=[G[u][v]['weight'] for u,v in edges]
        weights.append([i * 1e6 for i in (weights1)])
    maxWeightsNLT.append(np.max(np.hstack(weights)))
    minWeightsNLT.append(np.min(np.hstack(weights)))
    
    weightsMC=[]
    weights1MC=[]
    for j in range(len(weightedSubGraph[i])):
        maxNW=np.argmax(MC[j][i])
        minNW=np.argmin(MC[j][i])
        G=weightedSubGraph[i][j]
        edges=G.edges()
        weights1MC.append([G[u][v]['weight'] for u,v in edges])
        weightsMC.append([i * 1e6 for i in (weights1MC[j])])
    if weightsMC[0]:
        maxWeightsMC.append(np.max(np.array(weightsMC)))
        minWeightsMC.append(np.min(np.array(weightsMC)))

# minWeightsNLT=np.min(minWeightsNLT)
# maxWeightsNLT=np.max(maxWeightsNLT)

In [ ]:
#### PLOT SUBGRAPHS + CURRENT OVERLAY
#Max NLT
fig, axes = plt.subplots(11,len(NLT[0]),figsize=(10,30))
for i in tqdm(range(len(weightedSubGraph))): #for each network
    for j in range(len(weightedSubGraph[i])):#For each voltage

            #find top + bottom network
        ax=axes[j][i]
        G=weightedSubGraph[i][j]#current_new[i][j][maxNW]
        edge_weights=nx.get_edge_attributes(G,'weight')
        G.remove_edges_from((e for e, w in edge_weights.items() if w <1e-6)) 
        edges=G.edges()
    #         set_trace()
        weights=[G[u][v]['weight'] for u,v in edges]
#         weights=[w * 1e6 for w in (weights)]
#         cNorm=clrs.SymLogNorm(10,vmin=minWeightsNLT,vmax=maxWeightsNLT)
#         scalarMap = plt.cm.ScalarMappable(norm=cNorm, cmap=plt.cm.inferno)
#         colorList=[]
#         for k in range(len(weights)):
#             colorVal = scalarMap.to_rgba(weights[k])
#             colorList.append(colorVal)

        OGgraph=graphs4[i]
        #draw OG graph
        if i==1:
            top = nx.bipartite.sets(OGgraph)[0]
            pos=nx.bipartite_layout(OGgraph,top)
        else:
            pos=nx.kamada_kawai_layout(OGgraph)
        h=nx.draw_networkx_nodes(OGgraph,pos=pos,node_color='grey',node_size=10,ax=ax)
        h.set_zorder(1)

        if i == 0:
            ax.set_ylabel(str(onAmp[j])+'V')
        if j == 0:
            ax.set_title(nwNames4[i])
        h2=nx.draw_networkx_edges(G,pos=pos,ax=ax,edge_color=weights,edge_cmap=plt.cm.inferno,edge_vmin=np.min(np.array(weights1)),edge_vmax=np.max(np.array(weights1)))
    
        if h2:
            h2.set_norm(clrs.SymLogNorm(10))
            h2.set_zorder(3)
#             if j == 10 and i == 6:
#                 plt.colorbar(h2)
        nx.draw_networkx_nodes(G,pos=pos,nodelist=sources[i][j],node_color='#3f9b0b',node_size=120,node_shape ='*',ax=ax)
        nx.draw_networkx_nodes(G,pos=pos,nodelist=drains[i][j],node_color='#7e1e9c',node_size=120,node_shape ='*',ax=ax)
        ax.set_frame_on(False)
           
        ax.annotate('NLT = '+"{0:.2f}".format(NLT[j][i]),xy=(0.5,1),xycoords='axes fraction')
        ax.annotate('MC = '+"{0:.2f}".format(MC[j][i]),xy=(0.5,0.9),xycoords='axes fraction')
        
        ax.set_yticks([])
        ax.set_xticks([])

#     plt.savefig(saveFig+'VSweep 300NWN NetworkComparison_Elegans_NWN_ER_Crossbar - Current + Threshold Weighted Graphs NLT.pdf',format='pdf',dpi=300)

### Graph Theory For Weighted Network

In [ ]:
ciElegansWeighted=[None]*11;qElegansWeighted=[None]*11;pcoeffElegansWeighted=[None]*11;mzElegansWeighted=[None]*11
clusteringElegansWeighted=[None]*11;degElegansWeighted=[None]*11;avgDegElegansWeighted=[None]*11;SPLElegansWeighted=[None]*11
avgSPLElegansWeighted=[None]*11;localEfficiencyElegansWeighted=[None]*11;globalEfficiencyElegansWeighted=[None]*11
avgClusteringElegansWeighted=[None]*11

ciSparseCrossbarWeighted=[None]*11;qSparseCrossbarWeighted=[None]*11;pcoeffSparseCrossbarWeighted=[None]*11;mzSparseCrossbarWeighted=[None]*11
clusteringSparseCrossbarWeighted=[None]*11;degSparseCrossbarWeighted=[None]*11;avgDegSparseCrossbarWeighted=[None]*11
SPLSparseCrossbarWeighted=[None]*11;avgSPLSparseCrossbarWeighted=[None]*11;localEfficiencySparseCrossbarWeighted=[None]*11;globalEfficiencySparseCrossbarWeighted=[None]*11
avgClusteringSparseCrossbarWeighted=[None]*11

ciNWNWeighted=[None]*11;qNWNWeighted=[None]*11;pcoeffNWNWeighted=[None]*11;mzNWNWeighted=[None]*11
clusteringNWNWeighted=[None]*11;degNWNWeighted=[None]*11;avgDegNWNWeighted=[None]*11;SPLNWNWeighted=[None]*11;
avgSPLNWNWeighted=[None]*11;localEfficiencyNWNWeighted=[None]*11;globalEfficiencyNWNWeighted=[None]*11
avgClusteringNWNWeighted=[None]*11

ciRewiredWeighted=[None]*11;qRewiredWeighted=[None]*11;pcoeffRewiredWeighted=[None]*11;mzRewiredWeighted=[None]*11
clusteringRewiredWeighted=[None]*11;degRewiredWeighted=[None]*11;avgDegRewiredWeighted=[None]*11;SPLRewiredWeighted=[None]*11;
avgSPLRewiredWeighted=[None]*11;localEfficiencyRewiredWeighted=[None]*11;globalEfficiencyRewiredWeighted=[None]*11
avgClusteringRewiredWeighted=[None]*11

for i in tqdm(range(len(onAmp))):
    ciElegansWeighted[i],qElegansWeighted[i]=bct.community_louvain(np.array(nx.adjacency_matrix(weightedSubGraph[0][i]).todense()),gamma=1.1)
    pcoeffElegansWeighted[i]=bct.participation_coef(np.squeeze(np.array(nx.adjacency_matrix(weightedSubGraph[0][i]).todense())),ciElegansWeighted[i])
    mzElegansWeighted[i]=bct.module_degree_zscore(np.squeeze(np.array(nx.adjacency_matrix(weightedSubGraph[0][i]).todense())),ciElegansWeighted[i])
    clusteringElegansWeighted[i]=nx.clustering(weightedSubGraph[0][i])
    avgClusteringElegansWeighted[i]=np.mean(list(clusteringElegansWeighted[i].values()))
    degElegansWeighted[i]=[val for (nodes,val) in weightedSubGraph[0][i].degree()]
    avgDegElegansWeighted[i]=np.mean(degElegansWeighted[i])
    AdjMat=nx.adjacency_matrix(weightedSubGraph[0][i]).todense()
#     SPLElegansWeighted[i]=nx.shortest_path_length(weightedSubGraph[0][i],source=getFarthestPairing(AdjMat)[0],target=getFarthestPairing(AdjMat)[1])
#     avgSPLElegansWeighted[i]=nx.average_shortest_path_length(weightedSubGraph[0][i])
    localEfficiencyElegansWeighted[i]=nx.local_efficiency(weightedSubGraph[0][i])
    globalEfficiencyElegansWeighted[i]=nx.global_efficiency(weightedSubGraph[0][i])

    ciSparseCrossbarWeighted[i],qSparseCrossbarWeighted[i]=bct.community_louvain(np.array(nx.adjacency_matrix(weightedSubGraph[1][i]).todense()),gamma=1.1)
    pcoeffSparseCrossbarWeighted[i]=bct.participation_coef(np.squeeze(np.array(nx.adjacency_matrix(weightedSubGraph[1][i]).todense())),ciSparseCrossbarWeighted[i])
    mzSparseCrossbarWeighted[i]=bct.module_degree_zscore(np.squeeze(np.array(nx.adjacency_matrix(weightedSubGraph[1][i]).todense())),ciSparseCrossbarWeighted[i])
    clusteringSparseCrossbarWeighted[i]=nx.clustering(weightedSubGraph[1][i])
    avgClusteringSparseCrossbarWeighted[i]=np.mean(list(clusteringSparseCrossbarWeighted[i].values()))
    degSparseCrossbarWeighted[i]=[val for (nodes,val) in weightedSubGraph[1][i].degree()]
    avgDegSparseCrossbarWeighted[i]=np.mean(degSparseCrossbarWeighted[i])
    AdjMat=nx.adjacency_matrix(weightedSubGraph[1][i]).todense()
#     SPLElegansWeighted[i]=nx.shortest_path_length(weightedSubGraph[0][i],source=getFarthestPairing(AdjMat)[0],target=getFarthestPairing(AdjMat)[1])
#     avgSPLElegansWeighted[i]=nx.average_shortest_path_length(weightedSubGraph[0][i])
    localEfficiencySparseCrossbarWeighted[i]=nx.local_efficiency(weightedSubGraph[1][i])
    globalEfficiencySparseCrossbarWeighted[i]=nx.global_efficiency(weightedSubGraph[1][i])

    ciNWNWeighted[i],qNWNWeighted[i]=bct.community_louvain(np.array(nx.adjacency_matrix(weightedSubGraph[2][i]).todense()),gamma=1.1)
    pcoeffNWNWeighted[i]=bct.participation_coef(np.squeeze(np.array(nx.adjacency_matrix(weightedSubGraph[2][i]).todense())),ciNWNWeighted[i])
    mzNWNWeighted[i]=bct.module_degree_zscore(np.squeeze(np.array(nx.adjacency_matrix(weightedSubGraph[2][i]).todense())),ciNWNWeighted[i])
    clusteringNWNWeighted[i]=nx.clustering(weightedSubGraph[2][i])
    avgClusteringNWNWeighted[i]=np.mean(list(clusteringNWNWeighted[i].values()))
    degNWNWeighted[i]=[val for (nodes,val) in weightedSubGraph[2][i].degree()]
    avgDegNWNWeighted[i]=np.mean(degNWNWeighted[i])
    AdjMat=nx.adjacency_matrix(weightedSubGraph[2][i]).todense()
#     SPLElegansWeighted[i]=nx.shortest_path_length(weightedSubGraph[0][i],source=getFarthestPairing(AdjMat)[0],target=getFarthestPairing(AdjMat)[1])
#     avgSPLElegansWeighted[i]=nx.average_shortest_path_length(weightedSubGraph[0][i])
    localEfficiencyNWNWeighted[i]=nx.local_efficiency(weightedSubGraph[2][i])
    globalEfficiencyNWNWeighted[i]=nx.global_efficiency(weightedSubGraph[2][i])

    ciRewiredWeighted[i],qRewiredWeighted[i]=bct.community_louvain(np.array(nx.adjacency_matrix(weightedSubGraph[3][i]).todense()),gamma=1.1)
    pcoeffRewiredWeighted[i]=bct.participation_coef(np.squeeze(np.array(nx.adjacency_matrix(weightedSubGraph[3][i]).todense())),ciRewiredWeighted[i])
    mzRewiredWeighted[i]=bct.module_degree_zscore(np.squeeze(np.array(nx.adjacency_matrix(weightedSubGraph[3][i]).todense())),ciRewiredWeighted[i])
    clusteringRewiredWeighted[i]=nx.clustering(weightedSubGraph[3][i])
    avgClusteringRewiredWeighted[i]=np.mean(list(clusteringRewiredWeighted[i].values()))
    degRewiredWeighted[i]=[val for (nodes,val) in weightedSubGraph[3][i].degree()]
    avgDegRewiredWeighted[i]=np.mean(degRewiredWeighted[i])
    AdjMat=nx.adjacency_matrix(weightedSubGraph[3][i]).todense()
#     SPLElegansWeighted[i]=nx.shortest_path_length(weightedSubGraph[0][i],source=getFarthestPairing(AdjMat)[0],target=getFarthestPairing(AdjMat)[1])
#     avgSPLElegansWeighted[i]=nx.average_shortest_path_length(weightedSubGraph[0][i])
    localEfficiencyRewiredWeighted[i]=nx.local_efficiency(weightedSubGraph[3][i])
    globalEfficiencyRewiredWeighted[i]=nx.global_efficiency(weightedSubGraph[3][i])


In [ ]:
a=plt.scatter(qElegansWeighted,NLT.T[0])
b=plt.scatter(qSparseCrossbarWeighted,NLT.T[1])
c=plt.scatter(qNWNWeighted,NLT.T[2])
d=plt.scatter(qRewiredWeighted,NLT.T[3])
plt.ylabel('NLT')
plt.xlabel('Weighted Modularity')
plt.legend([a,b,c,d],nwNames4)

In [ ]:
a=plt.scatter(avgClusteringElegansWeighted,NLT.T[0])
b=plt.scatter(avgClusteringSparseCrossbarWeighted,NLT.T[1])
c=plt.scatter(avgClusteringNWNWeighted,NLT.T[2])
d=plt.scatter(avgClusteringRewiredWeighted,NLT.T[3])
plt.ylabel('NLT')
plt.xlabel('Weighted Avg Clustering')
plt.legend([a,b,c,d],nwNames4)

In [ ]:
a=plt.scatter(avgDegElegansWeighted,NLT.T[0])
b=plt.scatter(avgDegSparseCrossbarWeighted,NLT.T[1])
c=plt.scatter(avgDegNWNWeighted,NLT.T[2])
d=plt.scatter(avgDegRewiredWeighted,NLT.T[3])
plt.ylabel('NLT')
plt.xlabel('Weighted Avg Degree')
plt.legend([a,b,c,d],nwNames4)